In [ ]:
# 🔍 Colab NER 오류 진단 및 단계별 해결

# ==========================================
# 1단계: 현재 상태 진단
# ==========================================

print("🔍 === 현재 환경 진단 ===")

# 기본 정보 확인
import sys
print(f"Python 버전: {sys.version}")

try:
    import torch
    print(f"PyTorch 버전: {torch.__version__}")
    print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
        print(f"사용 중인 메모리: {torch.cuda.memory_allocated() / 1024**3:.1f}GB")
except ImportError as e:
    print(f"❌ PyTorch 오류: {e}")

try:
    import transformers
    print(f"Transformers 버전: {transformers.__version__}")
except ImportError as e:
    print(f"❌ Transformers 오류: {e}")

try:
    import datasets
    print(f"Datasets 버전: {datasets.__version__}")
except ImportError as e:
    print(f"❌ Datasets 오류: {e}")


🔍 === 현재 환경 진단 ===
Python 버전: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch 버전: 2.8.0+cu126
CUDA 사용 가능: True
GPU 메모리: 39.6GB
사용 중인 메모리: 0.0GB
Transformers 버전: 4.55.2
Datasets 버전: 4.0.0


In [ ]:
# ==========================================
# 2단계: 메모리 정리 및 재설치
# ==========================================

def clean_install():
    """완전 정리 후 재설치"""
    print("\n🧹 === 환경 정리 및 재설치 ===")

    # GPU 메모리 정리
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("✅ GPU 메모리 정리 완료")
    except:
        pass

    # 패키지 완전 삭제
    print("📦 기존 패키지 제거 중...")
    import subprocess

    packages_to_remove = [
        "transformers", "datasets", "tokenizers",
        "torch", "torchvision", "torchaudio",
        "accelerate", "evaluate", "seqeval"
    ]

    for package in packages_to_remove:
        try:
            subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", package],
                          capture_output=True, check=False)
        except:
            pass

    # 캐시 정리
    import os
    cache_dirs = [
        "~/.cache/huggingface/",
        "~/.cache/pip/",
        "/tmp/pip-*"
    ]

    for cache_dir in cache_dirs:
        try:
            os.system(f"rm -rf {cache_dir}")
        except:
            pass

    print("🔄 새로운 패키지 설치 중...")

    # 안정 버전으로 순차 설치
    install_commands = [
        "pip install --no-cache-dir torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118",
        "pip install --no-cache-dir transformers==4.30.2",
        "pip install --no-cache-dir datasets==2.12.0",
        "pip install --no-cache-dir tokenizers==0.13.3",
        "pip install --no-cache-dir accelerate==0.20.3",
        "pip install --no-cache-dir evaluate==0.4.0",
        "pip install --no-cache-dir seqeval==1.2.2"
    ]

    for cmd in install_commands:
        try:
            result = subprocess.run(cmd.split(), capture_output=True, text=True)
            if result.returncode != 0:
                print(f"⚠️ 설치 실패: {cmd}")
                print(f"오류: {result.stderr}")
            else:
                print(f"✅ 설치 완료: {cmd.split()[2]}")
        except Exception as e:
            print(f"❌ 설치 오류: {e}")

    print("\n⚠️ 런타임을 재시작해주세요! (Ctrl+M, .) 그 후 다음 단계를 실행하세요.")

In [ ]:
# ==========================================
# 3단계: 최소 테스트 (런타임 재시작 후 실행)
# ==========================================

def minimal_test():
    """최소한의 기능 테스트"""
    print("\n🧪 === 최소 기능 테스트 ===")

    try:
        # 기본 임포트 테스트
        import torch
        import transformers
        from transformers import AutoTokenizer, AutoModelForTokenClassification
        print("✅ 기본 임포트 성공")

        # 토크나이저 테스트
        tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
        print("✅ 토크나이저 로드 성공")

        # 간단한 토크나이징 테스트
        test_text = ["안녕", "하세요"]
        tokens = tokenizer(test_text, is_split_into_words=True, return_tensors="pt")
        print("✅ 토크나이징 성공")

        # 모델 로드 테스트 (작은 모델)
        model = AutoModelForTokenClassification.from_pretrained(
            "monologg/koelectra-base-v3-discriminator",
            num_labels=5  # 최소 라벨
        )
        print("✅ 모델 로드 성공")

        print("\n🎉 기본 설정 완료! 이제 학습을 시도할 수 있습니다.")
        return True

    except Exception as e:
        print(f"❌ 테스트 실패: {e}")
        import traceback
        print(f"상세 오류:\n{traceback.format_exc()}")
        return False


In [ ]:
# ==========================================
# 4단계: 샘플 데이터로 미니 학습 테스트
# ==========================================

def mini_training_test():
    """최소 데이터로 학습 테스트"""
    print("\n🚀 === 미니 학습 테스트 ===")

    try:
        import torch
        from transformers import (
            AutoTokenizer, AutoModelForTokenClassification,
            DataCollatorForTokenClassification, TrainingArguments, Trainer
        )
        from datasets import Dataset

        # 초소형 샘플 데이터
        sample_data = [
            {"tokens": ["고혈압", "진단"], "labels": [1, 0]},
            {"tokens": ["당뇨병", "치료"], "labels": [1, 0]},
            {"tokens": ["수술", "완료"], "labels": [0, 0]}
        ]

        dataset = Dataset.from_list(sample_data)

        # 라벨 설정
        label_names = ["O", "B-DISEASE"]
        label2id = {label: i for i, label in enumerate(label_names)}
        id2label = {i: label for label, i in label2id.items()}

        # 모델과 토크나이저
        tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
        model = AutoModelForTokenClassification.from_pretrained(
            "monologg/koelectra-base-v3-discriminator",
            num_labels=len(label_names),
            id2label=id2label,
            label2id=label2id
        )

        # 데이터 전처리
        def tokenize_function(examples):
            tokenized = tokenizer(
                examples["tokens"],
                is_split_into_words=True,
                truncation=True,
                padding=True,
                return_tensors="pt"
            )

            # 라벨 정렬
            labels = []
            for i, label in enumerate(examples["labels"]):
                word_ids = tokenized.word_ids(batch_index=i)
                label_ids = []
                for word_id in word_ids:
                    if word_id is None:
                        label_ids.append(-100)
                    else:
                        label_ids.append(label[word_id] if word_id < len(label) else 0)
                labels.append(label_ids)

            tokenized["labels"] = labels
            return tokenized

        dataset = dataset.map(tokenize_function, batched=True)

        # 초소형 학습 설정
        training_args = TrainingArguments(
            output_dir="./test-ner",
            num_train_epochs=1,  # 1 에포크만
            per_device_train_batch_size=1,  # 배치 크기 1
            logging_steps=1,
            save_steps=10,
            evaluation_strategy="no",  # 평가 없음
            save_strategy="no",  # 저장 없음
            report_to="none"
        )

        # 트레이너
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset,
            data_collator=DataCollatorForTokenClassification(tokenizer)
        )

        print("학습 시작...")
        trainer.train()
        print("✅ 미니 학습 성공!")

        return True

    except Exception as e:
        print(f"❌ 미니 학습 실패: {e}")
        import traceback
        print(f"상세 오류:\n{traceback.format_exc()}")
        return False

In [ ]:
# ==========================================
# 5단계: 문제 해결 가이드
# ==========================================

def troubleshooting_guide():
    """문제 해결 가이드"""
    print("\n📋 === 문제 해결 가이드 ===")

    common_errors = {
        "CUDA out of memory": [
            "배치 크기를 4 또는 2로 줄이기",
            "gradient_accumulation_steps 늘리기",
            "fp16=True 사용",
            "런타임을 GPU High-RAM으로 변경"
        ],
        "AttributeError": [
            "transformers 버전 확인 (4.30.2 권장)",
            "런타임 재시작 후 재설치",
            "패키지 간 버전 충돌 해결"
        ],
        "KeyError": [
            "데이터 형태 확인 (tokens, labels 컬럼)",
            "라벨 매핑 확인",
            "토크나이저 설정 점검"
        ],
        "ImportError": [
            "패키지 재설치",
            "의존성 충돌 해결",
            "Python 버전 확인"
        ]
    }

    print("일반적인 오류와 해결책:")
    for error, solutions in common_errors.items():
        print(f"\n🔸 {error}:")
        for solution in solutions:
            print(f"   - {solution}")

In [ ]:
# ==========================================
# 실행 순서 가이드
# ==========================================

print("""
🎯 === 단계별 실행 가이드 ===

1️⃣ 현재 상태 진단:
   - 위의 진단 결과를 확인하세요

2️⃣ 문제가 있다면:
   clean_install()  # 실행 후 런타임 재시작 필수!

3️⃣ 런타임 재시작 후:
   minimal_test()  # 기본 기능 테스트

4️⃣ 기본 테스트 성공 시:
   mini_training_test()  # 미니 학습 테스트

5️⃣ 모든 테스트 통과 시:
   실제 NER 학습 코드 실행

⚠️ 각 단계에서 오류 발생 시 오류 메시지를 알려주세요!
""")


🎯 === 단계별 실행 가이드 ===

1️⃣ 현재 상태 진단:
   - 위의 진단 결과를 확인하세요

2️⃣ 문제가 있다면:
   clean_install()  # 실행 후 런타임 재시작 필수!

3️⃣ 런타임 재시작 후:
   minimal_test()  # 기본 기능 테스트

4️⃣ 기본 테스트 성공 시:
   mini_training_test()  # 미니 학습 테스트

5️⃣ 모든 테스트 통과 시:
   실제 NER 학습 코드 실행

⚠️ 각 단계에서 오류 발생 시 오류 메시지를 알려주세요!



In [ ]:
# 이것만 먼저 실행하고 결과를 알려주세요
print("🔍 현재 환경 진단...")

import sys
print(f"Python: {sys.version}")

try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU 메모리: {torch.cuda.memory_allocated() / 1024**3:.1f}GB 사용 중")
except Exception as e:
    print(f"PyTorch 오류: {e}")

try:
    import transformers
    print(f"Transformers: {transformers.__version__}")
except Exception as e:
    print(f"Transformers 오류: {e}")

🔍 현재 환경 진단...
Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA: True
GPU 메모리: 0.0GB 사용 중
Transformers: 4.55.2


In [ ]:
# 🛡️ 단계별 NER 학습 (안전한 버전)
# 환경이 정상이므로 차근차근 진행합니다

import warnings
warnings.filterwarnings('ignore')

# ==========================================
# 1단계: KBMC 데이터셋 확인 및 로드
# ==========================================

print("📊 === 1단계: 데이터셋 확인 ===")

try:
    from datasets import load_dataset

    # KBMC 데이터셋 로드 시도
    print("KBMC 데이터셋 로드 중...")
    dataset = load_dataset("SungJoo/KBMC", trust_remote_code=True)

    print(f"✅ 데이터셋 로드 성공!")
    print(f"구조: {dataset}")

    # 샘플 데이터 확인
    sample = dataset['train'][0]
    print(f"샘플 데이터: {sample}")

    # 데이터 형태 분석
    print(f"컬럼명: {dataset['train'].column_names}")
    print(f"훈련 데이터 크기: {len(dataset['train'])}")

except Exception as e:
    print(f"❌ KBMC 로드 실패: {e}")
    print("🔄 샘플 데이터로 대체합니다...")

    # 샘플 의료 NER 데이터 생성
    from datasets import Dataset

    sample_data = [
        {
            "tokens": ["환자는", "고혈압", "진단을", "받았다", "."],
            "ner_tags": [0, 1, 0, 0, 0]  # O, B-DISEASE, O, O, O
        },
        {
            "tokens": ["당뇨병", "약물", "치료", "시작", "."],
            "ner_tags": [1, 3, 2, 0, 0]  # B-DISEASE, B-DRUG, B-PROCEDURE, O, O
        },
        {
            "tokens": ["심근경색", "수술", "후", "회복", "중", "."],
            "ner_tags": [1, 2, 0, 0, 0, 0]  # B-DISEASE, B-PROCEDURE, O, O, O, O
        },
        {
            "tokens": ["MRI", "검사", "결과", "정상", "."],
            "ner_tags": [2, 2, 0, 0, 0]  # B-PROCEDURE, I-PROCEDURE, O, O, O
        },
        {
            "tokens": ["인슐린", "주사", "투여", "."],
            "ner_tags": [3, 2, 2, 0]  # B-DRUG, B-PROCEDURE, I-PROCEDURE, O
        }
    ]

    dataset = {"train": Dataset.from_list(sample_data)}
    print(f"✅ 샘플 데이터 생성 완료: {len(sample_data)}개")

print("\n" + "="*50)


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'SungJoo/KBMC' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'SungJoo/KBMC' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


📊 === 1단계: 데이터셋 확인 ===
KBMC 데이터셋 로드 중...


README.md: 0.00B [00:00, ?B/s]

KBMC.tsv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/6150 [00:00<?, ? examples/s]

✅ 데이터셋 로드 성공!
구조: DatasetDict({
    train: Dataset({
        features: ['Sentence', 'Tags'],
        num_rows: 6150
    })
})
샘플 데이터: {'Sentence': '전신 적 다한증 은 신체 전체 에 힘 이 빠져서 일상 생활 이 어려워지는 질환 으로 , 근육 통증 과 무기 력 감 이 동반 됩니다 .', 'Tags': 'Disease-B Disease-I Disease-I O O O O O O O O O O O O O O Disease-B Disease-I O Disease-B Disease-I Disease-I O O O O'}
컬럼명: ['Sentence', 'Tags']
훈련 데이터 크기: 6150



In [ ]:
# ==========================================
# 2단계: 라벨 시스템 정의
# ==========================================

print("🏷️ === 2단계: 라벨 시스템 정의 ===")

# 의료 NER 라벨 정의
label_names = [
    "O",           # 0: Outside (일반 단어)
    "B-DISEASE",   # 1: 질병 시작
    "B-PROCEDURE", # 2: 시술/검사 시작
    "B-DRUG",      # 3: 약물 시작
    "I-DISEASE",   # 4: 질병 내부
    "I-PROCEDURE", # 5: 시술/검사 내부
    "I-DRUG",      # 6: 약물 내부
]

label2id = {label: i for i, label in enumerate(label_names)}
id2label = {i: label for label, i in label2id.items()}

print(f"라벨 개수: {len(label_names)}")
print(f"라벨 목록: {label_names}")

print("\n" + "="*50)

🏷️ === 2단계: 라벨 시스템 정의 ===
라벨 개수: 7
라벨 목록: ['O', 'B-DISEASE', 'B-PROCEDURE', 'B-DRUG', 'I-DISEASE', 'I-PROCEDURE', 'I-DRUG']



In [ ]:
# ==========================================
# 3단계: 토크나이저 로드 및 테스트
# ==========================================

print("🔤 === 3단계: 토크나이저 로드 ===")

try:
    from transformers import AutoTokenizer

    tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

    # 패딩 토큰 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("✅ 토크나이저 로드 성공")

    # 테스트 토크나이징
    test_tokens = ["고혈압", "진단"]
    test_result = tokenizer(test_tokens, is_split_into_words=True, return_tensors="pt")

    print(f"테스트 토큰: {test_tokens}")
    print(f"토크나이징 결과 형태: {test_result['input_ids'].shape}")
    print("✅ 토크나이징 테스트 성공")

except Exception as e:
    print(f"❌ 토크나이저 오류: {e}")
    raise e

print("\n" + "="*50)


🔤 === 3단계: 토크나이저 로드 ===


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

✅ 토크나이저 로드 성공
테스트 토큰: ['고혈압', '진단']
토크나이징 결과 형태: torch.Size([1, 6])
✅ 토크나이징 테스트 성공



In [ ]:
# ==========================================
# 4단계: 데이터 전처리 함수 정의
# ==========================================

print("⚙️ === 4단계: 데이터 전처리 함수 ===")

def tokenize_and_align_labels(examples):
    """토큰과 라벨 정렬"""

    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=False,  # 여기서는 패딩하지 않음
        max_length=512,
        return_overflowing_tokens=False
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # 특수 토큰 ([CLS], [SEP] 등)
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # 새로운 단어의 첫 번째 토큰
                if word_idx < len(label):
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(0)  # 범위 초과 시 O 태그
            else:
                # 같은 단어의 후속 서브워드 토큰
                if word_idx < len(label):
                    current_label = label[word_idx]
                    # B- 태그를 I- 태그로 변환
                    if current_label in [1, 2, 3]:  # B- 태그들
                        label_ids.append(current_label + 3)  # I- 태그로 변환
                    else:
                        label_ids.append(current_label)
                else:
                    label_ids.append(0)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 테스트 전처리
print("전처리 함수 테스트 중...")
try:
    test_sample = dataset["train"][:2]  # 첫 2개 샘플
    processed = tokenize_and_align_labels(test_sample)

    print(f"✅ 전처리 테스트 성공")
    print(f"입력 토큰 수: {len(processed['input_ids'][0])}")
    print(f"라벨 수: {len(processed['labels'][0])}")

except Exception as e:
    print(f"❌ 전처리 오류: {e}")
    raise e

print("\n" + "="*50)


⚙️ === 4단계: 데이터 전처리 함수 ===
전처리 함수 테스트 중...
❌ 전처리 오류: 'tokens'


KeyError: 'tokens'

In [ ]:
# 🔧 데이터 구조 문제 해결

print("🔍 === 데이터 구조 확인 및 수정 ===")

# ==========================================
# 1단계: 현재 데이터셋 구조 상세 분석
# ==========================================

print("현재 데이터셋 구조 분석 중...")

# dataset이 이미 로드되어 있다고 가정
try:
    if isinstance(dataset, dict):
        train_data = dataset["train"]
    else:
        train_data = dataset

    print(f"데이터셋 타입: {type(train_data)}")
    print(f"컬럼명: {train_data.column_names}")
    print(f"데이터 크기: {len(train_data)}")

    # 첫 번째 샘플 상세 분석
    first_sample = train_data[0]
    print(f"첫 번째 샘플: {first_sample}")
    print(f"샘플 키들: {first_sample.keys()}")

    # 각 키의 값 타입과 내용 확인
    for key, value in first_sample.items():
        print(f"  {key}: {type(value)} - {value}")

except Exception as e:
    print(f"데이터셋 분석 오류: {e}")

print("\n" + "="*50)


🔍 === 데이터 구조 확인 및 수정 ===
현재 데이터셋 구조 분석 중...
데이터셋 타입: <class 'datasets.arrow_dataset.Dataset'>
컬럼명: ['Sentence', 'Tags']
데이터 크기: 6150
첫 번째 샘플: {'Sentence': '전신 적 다한증 은 신체 전체 에 힘 이 빠져서 일상 생활 이 어려워지는 질환 으로 , 근육 통증 과 무기 력 감 이 동반 됩니다 .', 'Tags': 'Disease-B Disease-I Disease-I O O O O O O O O O O O O O O Disease-B Disease-I O Disease-B Disease-I Disease-I O O O O'}
샘플 키들: dict_keys(['Sentence', 'Tags'])
  Sentence: <class 'str'> - 전신 적 다한증 은 신체 전체 에 힘 이 빠져서 일상 생활 이 어려워지는 질환 으로 , 근육 통증 과 무기 력 감 이 동반 됩니다 .
  Tags: <class 'str'> - Disease-B Disease-I Disease-I O O O O O O O O O O O O O O Disease-B Disease-I O Disease-B Disease-I Disease-I O O O O



In [ ]:
# ==========================================
# 2단계: 올바른 컬럼명 찾기 및 데이터 정규화
# ==========================================

print("🔄 === 데이터 정규화 ===")

def normalize_dataset(dataset):
    """데이터셋을 표준 형태로 정규화"""

    if isinstance(dataset, dict):
        train_data = dataset["train"]
    else:
        train_data = dataset

    # 가능한 컬럼명들 확인
    columns = train_data.column_names
    print(f"사용 가능한 컬럼: {columns}")

    # 토큰 컬럼 찾기
    token_column = None
    for col in ['tokens', 'words', 'text', 'sentence', 'token']:
        if col in columns:
            token_column = col
            break

    # 라벨 컬럼 찾기
    label_column = None
    for col in ['ner_tags', 'labels', 'tags', 'ner_labels', 'pos_tags']:
        if col in columns:
            label_column = col
            break

    print(f"토큰 컬럼: {token_column}")
    print(f"라벨 컬럼: {label_column}")

    if not token_column or not label_column:
        print("⚠️ 적절한 컬럼을 찾을 수 없습니다. 수동으로 데이터를 생성합니다.")
        return create_manual_dataset()

    # 데이터 형태 확인
    sample = train_data[0]
    print(f"토큰 샘플: {sample[token_column]}")
    print(f"라벨 샘플: {sample[label_column]}")

    # 데이터를 표준 형태로 변환
    normalized_data = []

    for i in range(min(len(train_data), 100)):  # 처음 100개만 테스트
        item = train_data[i]

        tokens = item[token_column]
        labels = item[label_column]

        # 토큰이 문자열이면 분할
        if isinstance(tokens, str):
            tokens = tokens.split()

        # 라벨이 문자열이면 분할하고 숫자로 변환
        if isinstance(labels, str):
            labels = labels.split()
            # 문자 라벨을 숫자로 변환
            labels = [label2id.get(label, 0) for label in labels]
        elif isinstance(labels, list) and len(labels) > 0 and isinstance(labels[0], str):
            # 문자 라벨 리스트를 숫자로 변환
            labels = [label2id.get(label, 0) for label in labels]

        # 토큰과 라벨 길이 맞추기
        min_len = min(len(tokens), len(labels))
        tokens = tokens[:min_len]
        labels = labels[:min_len]

        if len(tokens) > 0 and len(labels) > 0:
            normalized_data.append({
                "tokens": tokens,
                "ner_tags": labels
            })

    from datasets import Dataset
    return Dataset.from_list(normalized_data)

def create_manual_dataset():
    """수동으로 의료 NER 데이터셋 생성"""
    print("🔧 수동 데이터셋 생성 중...")

    from datasets import Dataset

    manual_data = [
        {"tokens": ["환자는", "고혈압", "진단을", "받았다"], "ner_tags": [0, 1, 0, 0]},
        {"tokens": ["당뇨병", "약물", "치료", "시작"], "ner_tags": [1, 3, 2, 0]},
        {"tokens": ["심근경색", "수술", "후", "회복", "중"], "ner_tags": [1, 2, 0, 0, 0]},
        {"tokens": ["MRI", "검사", "결과", "정상"], "ner_tags": [2, 2, 0, 0]},
        {"tokens": ["인슐린", "주사", "투여"], "ner_tags": [3, 2, 2]},
        {"tokens": ["폐렴", "진단", "받고", "항생제", "처방"], "ner_tags": [1, 0, 0, 3, 0]},
        {"tokens": ["CT", "촬영", "예정"], "ner_tags": [2, 2, 0]},
        {"tokens": ["혈압약", "복용", "중"], "ner_tags": [3, 0, 0]},
        {"tokens": ["수술", "성공적으로", "완료"], "ner_tags": [2, 0, 0]},
        {"tokens": ["암", "치료", "시작"], "ner_tags": [1, 2, 0]},
        {"tokens": ["X-ray", "검사", "정상"], "ner_tags": [2, 2, 0]},
        {"tokens": ["아스피린", "처방", "받음"], "ner_tags": [3, 0, 0]},
        {"tokens": ["위내시경", "검사", "예약"], "ner_tags": [2, 2, 0]},
        {"tokens": ["고지혈증", "약물", "변경"], "ner_tags": [1, 3, 0]},
        {"tokens": ["수혈", "실시", "완료"], "ner_tags": [2, 0, 0]},
        {"tokens": ["관절염", "치료", "지속"], "ner_tags": [1, 2, 0]},
        {"tokens": ["초음파", "검사", "시행"], "ner_tags": [2, 2, 0]},
        {"tokens": ["진통제", "투약", "중"], "ner_tags": [3, 2, 0]},
        {"tokens": ["백내장", "수술", "예정"], "ner_tags": [1, 2, 0]},
        {"tokens": ["혈액검사", "결과", "확인"], "ner_tags": [2, 0, 0]}
    ]

    print(f"✅ 수동 데이터셋 생성 완료: {len(manual_data)}개 샘플")
    return Dataset.from_list(manual_data)

# 데이터 정규화 실행
try:
    normalized_dataset = normalize_dataset(dataset)
    print(f"✅ 데이터 정규화 성공: {len(normalized_dataset)}개 샘플")

    # 정규화된 데이터 확인
    sample = normalized_dataset[0]
    print(f"정규화된 샘플: {sample}")

except Exception as e:
    print(f"❌ 정규화 실패: {e}")
    print("수동 데이터셋을 생성합니다...")
    normalized_dataset = create_manual_dataset()

print("\n" + "="*50)

🔄 === 데이터 정규화 ===
사용 가능한 컬럼: ['Sentence', 'Tags']
토큰 컬럼: None
라벨 컬럼: None
⚠️ 적절한 컬럼을 찾을 수 없습니다. 수동으로 데이터를 생성합니다.
🔧 수동 데이터셋 생성 중...
✅ 수동 데이터셋 생성 완료: 20개 샘플
✅ 데이터 정규화 성공: 20개 샘플
정규화된 샘플: {'tokens': ['환자는', '고혈압', '진단을', '받았다'], 'ner_tags': [0, 1, 0, 0]}



In [ ]:
# 🎯 KBMC 데이터셋 정확한 전처리

print("🔄 === KBMC 데이터셋 정규화 ===")

# ==========================================
# 1단계: KBMC 라벨 분석 및 매핑
# ==========================================

# 모든 라벨 추출하여 분석
all_tags = set()
for i in range(min(1000, len(dataset["train"]))):  # 처음 1000개 샘플에서 라벨 추출
    tags = dataset["train"][i]["Tags"].split()
    all_tags.update(tags)

print(f"발견된 라벨들: {sorted(all_tags)}")

# KBMC 라벨 시스템 정의 (실제 발견된 라벨 기반)
kbmc_labels = sorted(list(all_tags))
print(f"총 라벨 수: {len(kbmc_labels)}")

# 라벨 매핑 생성
label2id = {label: i for i, label in enumerate(kbmc_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"라벨 매핑:")
for label, id_num in label2id.items():
    print(f"  {label} -> {id_num}")

print("\n" + "="*50)


🔄 === KBMC 데이터셋 정규화 ===
발견된 라벨들: ['Body-B', 'Body-I', 'Disease-B', 'Disease-I', 'O', 'Treatment-B', 'Treatment-I']
총 라벨 수: 7
라벨 매핑:
  Body-B -> 0
  Body-I -> 1
  Disease-B -> 2
  Disease-I -> 3
  O -> 4
  Treatment-B -> 5
  Treatment-I -> 6



In [ ]:
# ==========================================
# 2단계: 데이터 변환 함수
# ==========================================

def convert_kbmc_to_standard(sample):
    """KBMC 형태를 표준 NER 형태로 변환"""

    # 문장을 토큰으로 분할 (공백 기준)
    sentence = sample["Sentence"]
    tokens = sentence.split()

    # 태그를 분할
    tags_str = sample["Tags"]
    tags = tags_str.split()

    # 토큰과 태그 길이 맞추기
    min_len = min(len(tokens), len(tags))
    tokens = tokens[:min_len]
    tags = tags[:min_len]

    # 태그를 ID로 변환
    tag_ids = [label2id[tag] for tag in tags]

    return {
        "tokens": tokens,
        "ner_tags": tag_ids
    }

# 몇 개 샘플로 변환 테스트
print("🧪 변환 테스트:")
for i in range(3):
    original = dataset["train"][i]
    converted = convert_kbmc_to_standard(original)

    print(f"\n샘플 {i+1}:")
    print(f"원본 문장: {original['Sentence'][:50]}...")
    print(f"토큰 수: {len(converted['tokens'])}")
    print(f"태그 수: {len(converted['ner_tags'])}")
    print(f"첫 5개 토큰: {converted['tokens'][:5]}")
    print(f"첫 5개 태그: {converted['ner_tags'][:5]}")

print("\n" + "="*50)

🧪 변환 테스트:

샘플 1:
원본 문장: 전신 적 다한증 은 신체 전체 에 힘 이 빠져서 일상 생활 이 어려워지는 질환 으로 , 근...
토큰 수: 27
태그 수: 27
첫 5개 토큰: ['전신', '적', '다한증', '은', '신체']
첫 5개 태그: [2, 3, 3, 4, 4]

샘플 2:
원본 문장: 췌장암 이란 췌장 에 생긴 암세포 로 이루어진 종괴 ( 종양 덩어리 ) 이다 ....
토큰 수: 15
태그 수: 15
첫 5개 토큰: ['췌장암', '이란', '췌장', '에', '생긴']
첫 5개 태그: [2, 4, 0, 4, 4]

샘플 3:
원본 문장: 이러한 병명 은 폐 기능 저하 로 인한 호흡 곤란 기침 천식 발작 등 의 증상 을 유발 하...
토큰 수: 27
태그 수: 27
첫 5개 토큰: ['이러한', '병명', '은', '폐', '기능']
첫 5개 태그: [4, 4, 4, 2, 3]



In [ ]:
# ==========================================
# 3단계: 전체 데이터셋 변환
# ==========================================

print("📊 === 전체 데이터셋 변환 ===")

from datasets import Dataset

# 전체 데이터셋 변환
print("전체 데이터 변환 중...")
converted_data = []

for i in range(len(dataset["train"])):
    if i % 1000 == 0:
        print(f"진행률: {i}/{len(dataset['train'])} ({i/len(dataset['train'])*100:.1f}%)")

    try:
        converted = convert_kbmc_to_standard(dataset["train"][i])

        # 최소 길이 체크 (너무 짧은 문장 제외)
        if len(converted["tokens"]) >= 3 and len(converted["ner_tags"]) >= 3:
            converted_data.append(converted)

    except Exception as e:
        print(f"샘플 {i} 변환 실패: {e}")
        continue

print(f"✅ 변환 완료: {len(converted_data)}개 샘플")

# 새로운 데이터셋 생성
converted_dataset = Dataset.from_list(converted_data)
print(f"변환된 데이터셋 크기: {len(converted_dataset)}")

print("\n" + "="*50)

📊 === 전체 데이터셋 변환 ===
전체 데이터 변환 중...
진행률: 0/6150 (0.0%)
진행률: 1000/6150 (16.3%)
진행률: 2000/6150 (32.5%)
진행률: 3000/6150 (48.8%)
진행률: 4000/6150 (65.0%)
진행률: 5000/6150 (81.3%)
진행률: 6000/6150 (97.6%)
✅ 변환 완료: 6148개 샘플
변환된 데이터셋 크기: 6148



In [ ]:
# ==========================================
# 4단계: 토크나이저와 라벨 정렬
# ==========================================

print("🔤 === 토크나이저 정렬 ===")

def tokenize_and_align_labels_kbmc(examples):
    """KBMC 데이터용 토크나이저 및 라벨 정렬"""

    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=False,
        max_length=512,
        return_overflowing_tokens=False
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # 특수 토큰 ([CLS], [SEP] 등)
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # 새로운 단어의 첫 번째 토큰
                if word_idx < len(label):
                    label_ids.append(label[word_idx])
                else:
                    # O 태그 ID 찾기 (없으면 0)
                    o_tag_id = label2id.get('O', 0)
                    label_ids.append(o_tag_id)
            else:
                # 같은 단어의 후속 서브워드
                if word_idx < len(label):
                    current_label_id = label[word_idx]
                    current_label = id2label[current_label_id]

                    # B- 태그를 I- 태그로 변환
                    if current_label.endswith('-B'):
                        i_tag = current_label.replace('-B', '-I')
                        if i_tag in label2id:
                            label_ids.append(label2id[i_tag])
                        else:
                            label_ids.append(current_label_id)
                    else:
                        label_ids.append(current_label_id)
                else:
                    o_tag_id = label2id.get('O', 0)
                    label_ids.append(o_tag_id)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 전처리 테스트
print("전처리 함수 테스트 중...")
try:
    test_batch = converted_dataset[:3]
    processed_test = tokenize_and_align_labels_kbmc(test_batch)

    print(f"✅ 전처리 테스트 성공!")
    print(f"배치 크기: {len(processed_test['input_ids'])}")

    for i in range(len(processed_test['input_ids'])):
        print(f"샘플 {i+1}: 토큰 {len(processed_test['input_ids'][i])}개, 라벨 {len(processed_test['labels'][i])}개")

except Exception as e:
    print(f"❌ 전처리 테스트 실패: {e}")
    import traceback
    print(traceback.format_exc())
    raise e

print("\n" + "="*50)

🔤 === 토크나이저 정렬 ===
전처리 함수 테스트 중...
✅ 전처리 테스트 성공!
배치 크기: 3
샘플 1: 토큰 33개, 라벨 33개
샘플 2: 토큰 20개, 라벨 20개
샘플 3: 토큰 33개, 라벨 33개



In [ ]:
# ==========================================
# 5단계: 전체 데이터셋 전처리
# ==========================================

print("⚙️ === 전체 데이터셋 전처리 ===")

# 학습/검증 분할
from sklearn.model_selection import train_test_split

# 80:20 분할
train_indices, val_indices = train_test_split(
    range(len(converted_dataset)),
    test_size=0.2,
    random_state=42
)

train_dataset = converted_dataset.select(train_indices)
val_dataset = converted_dataset.select(val_indices)

print(f"학습 데이터: {len(train_dataset)}개")
print(f"검증 데이터: {len(val_dataset)}개")

# 전처리 적용
print("학습 데이터 전처리 중...")
train_dataset_processed = train_dataset.map(
    tokenize_and_align_labels_kbmc,
    batched=True,
    batch_size=16,
    remove_columns=train_dataset.column_names,
    desc="학습 데이터 전처리"
)

print("검증 데이터 전처리 중...")
val_dataset_processed = val_dataset.map(
    tokenize_and_align_labels_kbmc,
    batched=True,
    batch_size=16,
    remove_columns=val_dataset.column_names,
    desc="검증 데이터 전처리"
)

print(f"✅ 전처리 완료!")
print(f"학습 데이터: {len(train_dataset_processed)}개")
print(f"검증 데이터: {len(val_dataset_processed)}개")

print("\n" + "="*50)

# ==========================================
# 6단계: 라벨 분포 분석
# ==========================================

print("📈 === 라벨 분포 분석 ===")

from collections import Counter

# 라벨 분포 계산
label_counts = Counter()
for sample in converted_data:
    for tag_id in sample["ner_tags"]:
        label_counts[tag_id] += 1

print("라벨 분포:")
for label_id, count in label_counts.most_common():
    label_name = id2label[label_id]
    percentage = count / sum(label_counts.values()) * 100
    print(f"  {label_name:15s}: {count:6d}개 ({percentage:5.1f}%)")

print(f"\n총 토큰 수: {sum(label_counts.values()):,}개")
print(f"고유 라벨 수: {len(label_counts)}개")

print("\n" + "="*50)

print("""
🎉 === KBMC 데이터 전처리 완료! ===

✅ 실제 KBMC 라벨 시스템 분석 완료
✅ Sentence/Tags 형태를 tokens/ner_tags로 변환 완료
✅ 전체 데이터셋 전처리 완료
✅ 학습/검증 분할 완료
✅ 라벨 분포 분석 완료

다음 단계: 모델 학습 시작!

전역 변수 설정:
- train_dataset_processed: 학습용 데이터
- val_dataset_processed: 검증용 데이터
- label2id, id2label: 라벨 매핑
- kbmc_labels: 전체 라벨 리스트
""")

# 전역 변수 설정
globals()['train_dataset_processed'] = train_dataset_processed
globals()['val_dataset_processed'] = val_dataset_processed
globals()['label2id'] = label2id
globals()['id2label'] = id2label
globals()['kbmc_labels'] = kbmc_labels

print("✅ 모든 변수가 전역으로 설정되었습니다!")

⚙️ === 전체 데이터셋 전처리 ===
학습 데이터: 4918개
검증 데이터: 1230개
학습 데이터 전처리 중...


학습 데이터 전처리:   0%|          | 0/4918 [00:00<?, ? examples/s]

검증 데이터 전처리 중...


검증 데이터 전처리:   0%|          | 0/1230 [00:00<?, ? examples/s]

✅ 전처리 완료!
학습 데이터: 4918개
검증 데이터: 1230개

📈 === 라벨 분포 분석 ===
라벨 분포:
  O              : 124879개 ( 81.1%)
  Disease-B      :  10595개 (  6.9%)
  Disease-I      :  10089개 (  6.6%)
  Body-B         :   5215개 (  3.4%)
  Treatment-B    :   1193개 (  0.8%)
  Body-I         :   1158개 (  0.8%)
  Treatment-I    :    839개 (  0.5%)

총 토큰 수: 153,968개
고유 라벨 수: 7개


🎉 === KBMC 데이터 전처리 완료! ===

✅ 실제 KBMC 라벨 시스템 분석 완료
✅ Sentence/Tags 형태를 tokens/ner_tags로 변환 완료
✅ 전체 데이터셋 전처리 완료
✅ 학습/검증 분할 완료
✅ 라벨 분포 분석 완료

다음 단계: 모델 학습 시작!

전역 변수 설정:
- train_dataset_processed: 학습용 데이터
- val_dataset_processed: 검증용 데이터  
- label2id, id2label: 라벨 매핑
- kbmc_labels: 전체 라벨 리스트

✅ 모든 변수가 전역으로 설정되었습니다!


In [ ]:
# 🔧 필요한 모듈 설치 후 NER 학습

print("📦 === 필요한 모듈 설치 ===")

# evaluate 및 seqeval 설치
import subprocess
import sys

def install_package(package):
    """패키지 설치"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        print(f"✅ {package} 설치 완료")
        return True
    except Exception as e:
        print(f"❌ {package} 설치 실패: {e}")
        return False

# 필요한 패키지들 설치
packages = ["evaluate", "seqeval"]
for package in packages:
    install_package(package)

print("\n" + "="*50)

📦 === 필요한 모듈 설치 ===
✅ evaluate 설치 완료
✅ seqeval 설치 완료



In [ ]:
# ==========================================
# NER 학습 (evaluate 없이도 작동하는 버전)
# ==========================================

print("🤖 === NER 모델 학습 시작 ===")

import torch
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


🤖 === NER 모델 학습 시작 ===


In [ ]:
# ==========================================
# 1단계: 모델 재로드
# ==========================================

print(f"🔧 모델 설정 중... (라벨 수: {len(kbmc_labels)})")

# 기존 모델 삭제 (메모리 절약)
if 'model' in globals():
    del model
    torch.cuda.empty_cache()

# 새 모델 로드
model = AutoModelForTokenClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=len(kbmc_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print(f"✅ 모델 로드 완료 (라벨 수: {model.num_labels})")

# GPU로 이동
if torch.cuda.is_available():
    model = model.cuda()
    print("✅ 모델을 GPU로 이동 완료")

print("\n" + "="*50)

🔧 모델 설정 중... (라벨 수: 7)


pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 로드 완료 (라벨 수: 7)
✅ 모델을 GPU로 이동 완료



In [ ]:
# ==========================================
# 2단계: 평가 함수 (fallback 버전)
# ==========================================

print("📊 === 평가 함수 설정 ===")

def compute_metrics_fallback(eval_pred):
    """evaluate 없이 작동하는 평가 함수"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # -100이 아닌 실제 라벨만 추출
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100:
                true_predictions.append(p)
                true_labels.append(l)

    # 기본 지표 계산
    accuracy = accuracy_score(true_labels, true_predictions)

    try:
        precision, recall, f1, _ = precision_recall_fscore_support(
            true_labels, true_predictions, average='weighted', zero_division=0
        )

        return {
            "accuracy": accuracy,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }
    except:
        return {"accuracy": accuracy}

# seqeval이 설치되었는지 확인 후 고급 평가 함수 사용
def compute_metrics_advanced(eval_pred):
    """seqeval을 사용한 고급 평가"""
    try:
        import evaluate
        seqeval = evaluate.load("seqeval")

        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=2)

        # 실제 토큰만 추출
        true_predictions = []
        true_labels = []

        for prediction, label in zip(predictions, labels):
            true_pred = []
            true_label = []

            for p, l in zip(prediction, label):
                if l != -100:
                    true_pred.append(id2label[p])
                    true_label.append(id2label[l])

            if len(true_pred) > 0:
                true_predictions.append(true_pred)
                true_labels.append(true_label)

        results = seqeval.compute(
            predictions=true_predictions,
            references=true_labels
        )

        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

    except Exception as e:
        print(f"고급 평가 실패, 기본 평가 사용: {e}")
        return compute_metrics_fallback(eval_pred)

# 평가 함수 선택
try:
    import evaluate
    compute_metrics = compute_metrics_advanced
    print("✅ 고급 평가 함수 (seqeval) 사용")
except:
    compute_metrics = compute_metrics_fallback
    print("✅ 기본 평가 함수 사용")

print("\n" + "="*50)

📊 === 평가 함수 설정 ===
✅ 고급 평가 함수 (seqeval) 사용



In [ ]:
# ==========================================
# 3단계: 학습 설정
# ==========================================

print("⚙️ === 학습 설정 ===")

# 데이터 콜레이터
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    max_length=512,
    pad_to_multiple_of=8,
    return_tensors="pt"
)

# 학습 인자 (안전한 설정)
training_args = TrainingArguments(
    output_dir="./kbmc-ner-koelectra",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # 메모리 부족 방지
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # 실제 배치 크기 = 4*4 = 16
    warmup_steps=300,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    dataloader_num_workers=0,  # 안정성을 위해 0
    fp16=True,
    dataloader_pin_memory=False,  # 메모리 문제 방지
    report_to="none",
    remove_unused_columns=False,
    push_to_hub=False,
)

print(f"✅ 학습 설정 완료")
print(f"   - 에포크: {training_args.num_train_epochs}")
print(f"   - 배치 크기: {training_args.per_device_train_batch_size}")
print(f"   - 실제 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

print("\n" + "="*50)

⚙️ === 학습 설정 ===


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# 🔧 필요한 모듈 설치 후 NER 학습

print("📦 === 필요한 모듈 설치 ===")

# evaluate 및 seqeval 설치
import subprocess
import sys

def install_package(package):
    """패키지 설치"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        print(f"✅ {package} 설치 완료")
        return True
    except Exception as e:
        print(f"❌ {package} 설치 실패: {e}")
        return False

# 필요한 패키지들 설치
packages = ["evaluate", "seqeval"]
for package in packages:
    install_package(package)

print("\n" + "="*50)

# ==========================================
# NER 학습 (evaluate 없이도 작동하는 버전)
# ==========================================

print("🤖 === NER 모델 학습 시작 ===")

import torch
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ==========================================
# 1단계: 모델 재로드
# ==========================================

print(f"🔧 모델 설정 중... (라벨 수: {len(kbmc_labels)})")

# 기존 모델 삭제 (메모리 절약)
if 'model' in globals():
    del model
    torch.cuda.empty_cache()

# 새 모델 로드
model = AutoModelForTokenClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=len(kbmc_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print(f"✅ 모델 로드 완료 (라벨 수: {model.num_labels})")

# GPU로 이동
if torch.cuda.is_available():
    model = model.cuda()
    print("✅ 모델을 GPU로 이동 완료")

print("\n" + "="*50)

# ==========================================
# 2단계: 평가 함수 (fallback 버전)
# ==========================================

print("📊 === 평가 함수 설정 ===")

def compute_metrics_fallback(eval_pred):
    """evaluate 없이 작동하는 평가 함수"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # -100이 아닌 실제 라벨만 추출
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100:
                true_predictions.append(p)
                true_labels.append(l)

    # 기본 지표 계산
    accuracy = accuracy_score(true_labels, true_predictions)

    try:
        precision, recall, f1, _ = precision_recall_fscore_support(
            true_labels, true_predictions, average='weighted', zero_division=0
        )

        return {
            "accuracy": accuracy,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }
    except:
        return {"accuracy": accuracy}

# seqeval이 설치되었는지 확인 후 고급 평가 함수 사용
def compute_metrics_advanced(eval_pred):
    """seqeval을 사용한 고급 평가"""
    try:
        import evaluate
        seqeval = evaluate.load("seqeval")

        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=2)

        # 실제 토큰만 추출
        true_predictions = []
        true_labels = []

        for prediction, label in zip(predictions, labels):
            true_pred = []
            true_label = []

            for p, l in zip(prediction, label):
                if l != -100:
                    true_pred.append(id2label[p])
                    true_label.append(id2label[l])

            if len(true_pred) > 0:
                true_predictions.append(true_pred)
                true_labels.append(true_label)

        results = seqeval.compute(
            predictions=true_predictions,
            references=true_labels
        )

        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

    except Exception as e:
        print(f"고급 평가 실패, 기본 평가 사용: {e}")
        return compute_metrics_fallback(eval_pred)

# 평가 함수 선택
try:
    import evaluate
    compute_metrics = compute_metrics_advanced
    print("✅ 고급 평가 함수 (seqeval) 사용")
except:
    compute_metrics = compute_metrics_fallback
    print("✅ 기본 평가 함수 사용")

print("\n" + "="*50)

# ==========================================
# 3단계: 학습 설정
# ==========================================

print("⚙️ === 학습 설정 ===")

# 데이터 콜레이터
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    max_length=512,
    pad_to_multiple_of=8,
    return_tensors="pt"
)

# 학습 인자 (기본 설정 - 모든 버전 호환)
print("기본 학습 설정 적용 중...")

training_args = TrainingArguments(
    output_dir="./kbmc-ner-koelectra",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=300,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    fp16=True,
    remove_unused_columns=False,
)

print(f"✅ 학습 설정 완료")
print(f"   - 에포크: {training_args.num_train_epochs}")
print(f"   - 배치 크기: {training_args.per_device_train_batch_size}")
print(f"   - 실제 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

print("\n" + "="*50)

# ==========================================
# 4단계: Trainer 설정 및 학습
# ==========================================

print("🏃 === Trainer 설정 및 학습 시작 ===")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=val_dataset_processed,  # 검증 데이터 유지
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료")
print(f"학습 데이터: {len(train_dataset_processed):,}개")
print(f"검증 데이터: {len(val_dataset_processed):,}개")

# GPU 메모리 정리
torch.cuda.empty_cache()

print("\n🚀 학습 시작! (예상 시간: 30-60분)")

try:
    # 학습 시작
    trainer.train()

    print("\n🎉 학습 완료!")

    # ==========================================
    # 5단계: 최종 평가 및 저장
    # ==========================================

    print("\n📊 === 최종 평가 ===")

    eval_results = trainer.evaluate()
    print("최종 검증 결과:")
    for key, value in eval_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")

    # ==========================================
    # 6단계: 모델 저장
    # ==========================================

    print("\n💾 === 모델 저장 ===")

    model.save_pretrained("./kbmc-ner-final")
    tokenizer.save_pretrained("./kbmc-ner-final")

    # 라벨 매핑 저장
    import json
    with open("./kbmc-ner-final/label_mapping.json", "w", encoding="utf-8") as f:
        json.dump({
            "label2id": label2id,
            "id2label": {str(k): v for k, v in id2label.items()},
            "labels": kbmc_labels
        }, f, ensure_ascii=False, indent=2)

    print("✅ 모델 저장 완료: ./kbmc-ner-final/")

    # ==========================================
    # 7단계: 간단한 테스트
    # ==========================================

    print("\n🧪 === 모델 테스트 ===")

    def test_trained_model(text):
        """학습된 모델 테스트"""
        tokens = text.split()
        inputs = tokenizer(
            tokens,
            is_split_into_words=True,
            return_tensors="pt",
            truncation=True,
            padding=True
        )

        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_ids = predictions.argmax(dim=-1)

        # 결과 정리
        results = []
        word_ids = inputs.word_ids()
        processed_words = set()

        for i, word_id in enumerate(word_ids):
            if word_id is not None and word_id < len(tokens) and word_id not in processed_words:
                token = tokens[word_id]
                pred_id = predicted_ids[0][i].item()
                confidence = predictions[0][i].max().item()
                label = id2label[pred_id]

                results.append((token, label, confidence))
                processed_words.add(word_id)

        return results

    # 테스트 문장들
    test_sentences = [
        "환자는 고혈압 진단을 받았습니다",
        "당뇨병 치료를 위해 인슐린을 투여합니다",
        "MRI 검사 결과 이상 소견이 발견되었습니다",
        "폐렴 진단 받고 항생제 처방받았습니다"
    ]

    for i, sentence in enumerate(test_sentences, 1):
        print(f"\n테스트 {i}: {sentence}")
        results = test_trained_model(sentence)

        entities_found = False
        for token, label, confidence in results:
            if label != 'O':  # O 태그가 아닌 것만 출력
                print(f"  {token} -> {label} (신뢰도: {confidence:.3f})")
                entities_found = True

        if not entities_found:
            print("  (의료 개체명이 감지되지 않았습니다)")

    print("\n🎊 === NER 모델 학습 완료! ===")
    print("✅ 모델이 성공적으로 학습되고 저장되었습니다.")
    print("📁 저장 위치: ./kbmc-ner-final/")
    print("🔄 이제 이 모델을 사용하여 심평원 데이터에서 의료용어를 추출할 수 있습니다!")

except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n❌ GPU 메모리 부족!")
        print("\n🔧 해결 방법:")
        print("1. 런타임을 재시작하고 다음 설정으로 재실행:")
        print("   - per_device_train_batch_size=2")
        print("   - gradient_accumulation_steps=8")
        print("2. 또는 런타임을 GPU High-RAM으로 변경")

        torch.cuda.empty_cache()

    else:
        print(f"❌ 런타임 오류: {e}")

except Exception as e:
    print(f"❌ 학습 오류: {e}")
    import traceback
    print(traceback.format_exc())

📦 === 필요한 모듈 설치 ===
✅ evaluate 설치 완료
✅ seqeval 설치 완료

🤖 === NER 모델 학습 시작 ===
🔧 모델 설정 중... (라벨 수: 7)


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 로드 완료 (라벨 수: 7)
✅ 모델을 GPU로 이동 완료

📊 === 평가 함수 설정 ===
✅ 고급 평가 함수 (seqeval) 사용

⚙️ === 학습 설정 ===
기본 학습 설정 적용 중...
✅ 학습 설정 완료
   - 에포크: 3
   - 배치 크기: 4
   - 실제 배치 크기: 16

🏃 === Trainer 설정 및 학습 시작 ===
✅ Trainer 설정 완료
학습 데이터: 4,918개
검증 데이터: 1,230개

🚀 학습 시작! (예상 시간: 30-60분)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:


❌ 런타임 오류: 


In [ ]:
# 🚫 wandb 완전 비활성화 후 NER 학습

print("🚫 === wandb 완전 비활성화 ===")

# wandb 완전 비활성화
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

# wandb 모듈이 있다면 완전 비활성화
try:
    import wandb
    wandb.init(mode="disabled")
    print("✅ wandb 비활성화 완료")
except:
    print("✅ wandb 모듈 없음 - 문제없음")

print("\n" + "="*50)

# ==========================================
# NER 학습 (wandb 없는 깔끔한 버전)
# ==========================================

print("🤖 === NER 모델 학습 (wandb 없음) ===")

import torch
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
from sklearn.metrics import accuracy_score

# ==========================================
# 1단계: 모델 재로드
# ==========================================

print(f"🔧 모델 설정 중... (라벨 수: {len(kbmc_labels)})")

# GPU 메모리 정리
torch.cuda.empty_cache()

# 모델 로드
model = AutoModelForTokenClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=len(kbmc_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print(f"✅ 모델 로드 완료 (라벨 수: {model.num_labels})")

if torch.cuda.is_available():
    model = model.cuda()
    print("✅ 모델을 GPU로 이동 완료")

print("\n" + "="*50)

# ==========================================
# 2단계: 간단한 평가 함수
# ==========================================

def compute_metrics_simple(eval_pred):
    """간단한 평가 함수 (wandb 없음)"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # -100이 아닌 실제 라벨만 추출
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100:
                true_predictions.append(p)
                true_labels.append(l)

    # 정확도 계산
    accuracy = accuracy_score(true_labels, true_predictions)

    return {"accuracy": accuracy}

print("✅ 간단한 평가 함수 설정 완료")

print("\n" + "="*50)

# ==========================================
# 3단계: 학습 설정 (wandb 완전 제거)
# ==========================================

print("⚙️ === 학습 설정 (wandb 제거) ===")

# 데이터 콜레이터
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)

# 학습 인자 (wandb 완전 제거)
training_args = TrainingArguments(
    output_dir="./kbmc-ner-final",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=300,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=1,  # 저장 공간 절약
    fp16=True,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to=[],  # 빈 리스트 - 아무것도 리포트하지 않음
    logging_first_step=False,
    disable_tqdm=False,  # 진행 바 유지
)

print("✅ 학습 설정 완료 (wandb 완전 제거)")

print("\n" + "="*50)

# ==========================================
# 4단계: Trainer 설정 및 학습
# ==========================================

print("🏃 === 학습 시작 ===")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_simple,
    # eval_dataset는 일단 제거 - 학습에만 집중
)

print(f"학습 데이터: {len(train_dataset_processed):,}개")
print("🚀 학습 시작! (wandb 없이 깔끔하게)")

try:
    # 학습 실행
    train_result = trainer.train()

    print("\n🎉 학습 완료!")
    print(f"최종 loss: {train_result.training_loss:.4f}")

    # ==========================================
    # 5단계: 검증 데이터로 평가
    # ==========================================

    print("\n📊 === 검증 데이터 평가 ===")

    # 검증 데이터로 평가
    trainer.eval_dataset = val_dataset_processed
    eval_results = trainer.evaluate()

    print("검증 결과:")
    for key, value in eval_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")

    # ==========================================
    # 6단계: 모델 저장
    # ==========================================

    print("\n💾 === 모델 저장 ===")

    model.save_pretrained("./kbmc-ner-final")
    tokenizer.save_pretrained("./kbmc-ner-final")

    # 라벨 매핑 저장
    import json
    with open("./kbmc-ner-final/label_mapping.json", "w", encoding="utf-8") as f:
        json.dump({
            "label2id": label2id,
            "id2label": {str(k): v for k, v in id2label.items()},
            "labels": kbmc_labels
        }, f, ensure_ascii=False, indent=2)

    print("✅ 모델 저장 완료: ./kbmc-ner-final/")

    # ==========================================
    # 7단계: 모델 테스트
    # ==========================================

    print("\n🧪 === 모델 테스트 ===")

    def test_final_model(text):
        """최종 모델 테스트"""
        tokens = text.split()
        inputs = tokenizer(
            tokens,
            is_split_into_words=True,
            return_tensors="pt",
            truncation=True,
            padding=True
        )

        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_ids = predictions.argmax(dim=-1)

        # 결과 정리
        results = []
        word_ids = inputs.word_ids()
        processed_words = set()

        for i, word_id in enumerate(word_ids):
            if word_id is not None and word_id < len(tokens) and word_id not in processed_words:
                token = tokens[word_id]
                pred_id = predicted_ids[0][i].item()
                confidence = predictions[0][i].max().item()
                label = id2label[pred_id]

                results.append((token, label, confidence))
                processed_words.add(word_id)

        return results

    # 테스트 문장들
    test_sentences = [
        "환자는 고혈압 진단을 받았습니다",
        "당뇨병 치료를 위해 인슐린을 투여합니다",
        "MRI 검사 결과 이상 소견이 발견되었습니다",
        "급성 심근경색 수술 후 회복 중입니다"
    ]

    print("테스트 결과:")
    for i, sentence in enumerate(test_sentences, 1):
        print(f"\n{i}. {sentence}")
        results = test_final_model(sentence)

        medical_entities = [(token, label, conf) for token, label, conf in results if label != 'O']

        if medical_entities:
            for token, label, confidence in medical_entities:
                print(f"   → {token}: {label} (신뢰도: {confidence:.3f})")
        else:
            print("   → 의료 개체명 없음")

    print("\n🎊 === NER 모델 학습 성공! ===")
    print("✅ wandb 없이 깔끔하게 학습 완료")
    print("📁 모델 저장: ./kbmc-ner-final/")
    print("🔄 이제 심평원 데이터에서 의료용어 추출 가능!")

    # 전역 변수로 저장
    globals()['trained_model'] = model
    globals()['final_tokenizer'] = tokenizer

    return trainer

except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n❌ GPU 메모리 부족!")
        print("해결 방법: 배치 크기를 2로 줄이고 재실행")
        torch.cuda.empty_cache()
    else:
        print(f"❌ 런타임 오류: {e}")

except Exception as e:
    print(f"❌ 학습 오류: {e}")
    import traceback
    print(traceback.format_exc())

🚫 === wandb 완전 비활성화 ===


✅ wandb 비활성화 완료

🤖 === NER 모델 학습 (wandb 없음) ===
🔧 모델 설정 중... (라벨 수: 7)


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 로드 완료 (라벨 수: 7)
✅ 모델을 GPU로 이동 완료

✅ 간단한 평가 함수 설정 완료

⚙️ === 학습 설정 (wandb 제거) ===
✅ 학습 설정 완료 (wandb 완전 제거)

🏃 === 학습 시작 ===
학습 데이터: 4,918개
🚀 학습 시작! (wandb 없이 깔끔하게)


SyntaxError: 'return' outside function (ipython-input-1639038865.py, line 261)

In [ ]:
# 🚫 wandb 완전 비활성화 후 NER 학습

print("🚫 === wandb 완전 비활성화 ===")

# wandb 완전 비활성화
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

# wandb 모듈이 있다면 완전 비활성화
try:
    import wandb
    wandb.init(mode="disabled")
    print("✅ wandb 비활성화 완료")
except:
    print("✅ wandb 모듈 없음 - 문제없음")

print("\n" + "="*50)

# ==========================================
# NER 학습 (wandb 없는 깔끔한 버전)
# ==========================================

print("🤖 === NER 모델 학습 (wandb 없음) ===")

import torch
from transformers import (
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
from sklearn.metrics import accuracy_score

# ==========================================
# 1단계: 모델 재로드
# ==========================================

print(f"🔧 모델 설정 중... (라벨 수: {len(kbmc_labels)})")

# GPU 메모리 정리
torch.cuda.empty_cache()

# 모델 로드
model = AutoModelForTokenClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=len(kbmc_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

print(f"✅ 모델 로드 완료 (라벨 수: {model.num_labels})")

if torch.cuda.is_available():
    model = model.cuda()
    print("✅ 모델을 GPU로 이동 완료")

print("\n" + "="*50)

# ==========================================
# 2단계: 간단한 평가 함수
# ==========================================

def compute_metrics_simple(eval_pred):
    """간단한 평가 함수 (wandb 없음)"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # -100이 아닌 실제 라벨만 추출
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        for p, l in zip(prediction, label):
            if l != -100:
                true_predictions.append(p)
                true_labels.append(l)

    # 정확도 계산
    accuracy = accuracy_score(true_labels, true_predictions)

    return {"accuracy": accuracy}

print("✅ 간단한 평가 함수 설정 완료")

print("\n" + "="*50)

# ==========================================
# 3단계: 학습 설정 (wandb 완전 제거)
# ==========================================

print("⚙️ === 학습 설정 (wandb 제거) ===")

# 데이터 콜레이터
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)

# 학습 인자 (wandb 완전 제거)
training_args = TrainingArguments(
    output_dir="./kbmc-ner-final",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=300,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=1,  # 저장 공간 절약
    fp16=True,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    report_to=[],  # 빈 리스트 - 아무것도 리포트하지 않음
    logging_first_step=False,
    disable_tqdm=False,  # 진행 바 유지
)

print("✅ 학습 설정 완료 (wandb 완전 제거)")

print("\n" + "="*50)

# ==========================================
# 4단계: Trainer 설정 및 학습
# ==========================================

print("🏃 === 학습 시작 ===")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_simple,
    # eval_dataset는 일단 제거 - 학습에만 집중
)

print(f"학습 데이터: {len(train_dataset_processed):,}개")
print("🚀 학습 시작! (wandb 없이 깔끔하게)")

try:
    # 학습 실행
    train_result = trainer.train()

    print("\n🎉 학습 완료!")
    print(f"최종 loss: {train_result.training_loss:.4f}")

    # ==========================================
    # 5단계: 검증 데이터로 평가
    # ==========================================

    print("\n📊 === 검증 데이터 평가 ===")

    # 검증 데이터로 평가
    trainer.eval_dataset = val_dataset_processed
    eval_results = trainer.evaluate()

    print("검증 결과:")
    for key, value in eval_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")

    # ==========================================
    # 6단계: 모델 저장
    # ==========================================

    print("\n💾 === 모델 저장 ===")

    model.save_pretrained("./kbmc-ner-final")
    tokenizer.save_pretrained("./kbmc-ner-final")

    # 라벨 매핑 저장
    import json
    with open("./kbmc-ner-final/label_mapping.json", "w", encoding="utf-8") as f:
        json.dump({
            "label2id": label2id,
            "id2label": {str(k): v for k, v in id2label.items()},
            "labels": kbmc_labels
        }, f, ensure_ascii=False, indent=2)

    print("✅ 모델 저장 완료: ./kbmc-ner-final/")

    # ==========================================
    # 7단계: 모델 테스트
    # ==========================================

    print("\n🧪 === 모델 테스트 ===")

    def test_final_model(text):
        """최종 모델 테스트"""
        tokens = text.split()
        inputs = tokenizer(
            tokens,
            is_split_into_words=True,
            return_tensors="pt",
            truncation=True,
            padding=True
        )

        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_ids = predictions.argmax(dim=-1)

        # 결과 정리
        results = []
        word_ids = inputs.word_ids()
        processed_words = set()

        for i, word_id in enumerate(word_ids):
            if word_id is not None and word_id < len(tokens) and word_id not in processed_words:
                token = tokens[word_id]
                pred_id = predicted_ids[0][i].item()
                confidence = predictions[0][i].max().item()
                label = id2label[pred_id]

                results.append((token, label, confidence))
                processed_words.add(word_id)

        return results

    # 테스트 문장들
    test_sentences = [
        "환자는 고혈압 진단을 받았습니다",
        "당뇨병 치료를 위해 인슐린을 투여합니다",
        "MRI 검사 결과 이상 소견이 발견되었습니다",
        "급성 심근경색 수술 후 회복 중입니다"
    ]

    print("테스트 결과:")
    for i, sentence in enumerate(test_sentences, 1):
        print(f"\n{i}. {sentence}")
        results = test_final_model(sentence)

        medical_entities = [(token, label, conf) for token, label, conf in results if label != 'O']

        if medical_entities:
            for token, label, confidence in medical_entities:
                print(f"   → {token}: {label} (신뢰도: {confidence:.3f})")
        else:
            print("   → 의료 개체명 없음")

    print("\n🎊 === NER 모델 학습 성공! ===")
    print("✅ wandb 없이 깔끔하게 학습 완료")
    print("📁 모델 저장: ./kbmc-ner-final/")
    print("🔄 이제 심평원 데이터에서 의료용어 추출 가능!")

    # 전역 변수로 저장
    globals()['trained_model'] = model
    globals()['final_tokenizer'] = tokenizer

    print("✅ 모든 변수가 전역으로 저장되었습니다!")

except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n❌ GPU 메모리 부족!")
        print("해결 방법: 배치 크기를 2로 줄이고 재실행")
        torch.cuda.empty_cache()
    else:
        print(f"❌ 런타임 오류: {e}")

except Exception as e:
    print(f"❌ 학습 오류: {e}")
    import traceback
    print(traceback.format_exc())

🚫 === wandb 완전 비활성화 ===


✅ wandb 비활성화 완료

🤖 === NER 모델 학습 (wandb 없음) ===
🔧 모델 설정 중... (라벨 수: 7)


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 로드 완료 (라벨 수: 7)
✅ 모델을 GPU로 이동 완료

✅ 간단한 평가 함수 설정 완료

⚙️ === 학습 설정 (wandb 제거) ===
✅ 학습 설정 완료 (wandb 완전 제거)

🏃 === 학습 시작 ===
학습 데이터: 4,918개
🚀 학습 시작! (wandb 없이 깔끔하게)


Step,Training Loss
100,1.006900
200,0.235100
300,0.160500
400,0.116900
500,0.106400
600,0.099800
700,0.075300
800,0.069000
900,0.068600



🎉 학습 완료!
최종 loss: 0.2113

📊 === 검증 데이터 평가 ===


검증 결과:
  eval_loss: 0.1032
  eval_accuracy: 0.9695
  eval_runtime: 5.1162
  eval_samples_per_second: 240.4110
  eval_steps_per_second: 60.2000
  epoch: 3.0000

💾 === 모델 저장 ===
✅ 모델 저장 완료: ./kbmc-ner-final/

🧪 === 모델 테스트 ===
테스트 결과:

1. 환자는 고혈압 진단을 받았습니다
❌ 학습 오류: 'dict' object has no attribute 'word_ids'
Traceback (most recent call last):
  File "/tmp/ipython-input-1252441118.py", line 242, in <cell line: 0>
    results = test_final_model(sentence)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1252441118.py", line 216, in test_final_model
    word_ids = inputs.word_ids()
               ^^^^^^^^^^^^^^^
AttributeError: 'dict' object has no attribute 'word_ids'



In [ ]:
# 💾 학습된 NER 모델 다운로드 및 백업

print("📦 === 학습된 모델 다운로드 준비 ===")

import os
import zipfile
from google.colab import files

# ==========================================
# 1단계: 모델 폴더 내용 확인
# ==========================================

print("📁 kbmc-ner-final 폴더 내용:")
if os.path.exists("./kbmc-ner-final"):
    for file in os.listdir("./kbmc-ner-final"):
        file_path = os.path.join("./kbmc-ner-final", file)
        size = os.path.getsize(file_path) / (1024*1024)  # MB 단위
        print(f"  {file} ({size:.1f} MB)")
else:
    print("❌ kbmc-ner-final 폴더가 없습니다!")

print("\n" + "="*50)

# ==========================================
# 2단계: 모델을 ZIP으로 압축
# ==========================================

print("🗜️ === 모델 압축 중 ===")

def create_model_zip():
    """모델 폴더를 ZIP 파일로 압축"""

    zip_filename = "kbmc-ner-model.zip"

    try:
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:

            # kbmc-ner-final 폴더의 모든 파일 추가
            for root, dirs, files in os.walk("./kbmc-ner-final"):
                for file in files:
                    file_path = os.path.join(root, file)
                    # ZIP 내에서의 경로 (kbmc-ner-final/ 포함)
                    arcname = os.path.relpath(file_path, ".")
                    zipf.write(file_path, arcname)
                    print(f"  추가: {arcname}")

        # ZIP 파일 크기 확인
        zip_size = os.path.getsize(zip_filename) / (1024*1024)
        print(f"\n✅ 압축 완료: {zip_filename} ({zip_size:.1f} MB)")

        return zip_filename

    except Exception as e:
        print(f"❌ 압축 실패: {e}")
        return None

zip_file = create_model_zip()

print("\n" + "="*50)

# ==========================================
# 3단계: 모델 사용법 텍스트 파일 생성
# ==========================================

print("📝 === 사용법 가이드 생성 ===")

usage_guide = """
🤖 KBMC NER 모델 사용법 가이드

이 모델은 한국어 의료 개체명 인식(NER)을 위해 KoELECTRA 기반으로 학습된 모델입니다.

📁 포함된 파일:
- config.json: 모델 설정
- pytorch_model.bin: 학습된 모델 가중치
- tokenizer.json: 토크나이저
- vocab.txt: 어휘 사전
- label_mapping.json: 라벨 매핑 정보

🚀 사용 방법:

1. 필요한 라이브러리 설치:
   pip install transformers torch

2. 모델 로드:
   from transformers import AutoTokenizer, AutoModelForTokenClassification
   import json

   # 모델과 토크나이저 로드
   model = AutoModelForTokenClassification.from_pretrained("./kbmc-ner-final")
   tokenizer = AutoTokenizer.from_pretrained("./kbmc-ner-final")

   # 라벨 매핑 로드
   with open("./kbmc-ner-final/label_mapping.json", "r", encoding="utf-8") as f:
       label_info = json.load(f)
   id2label = {int(k): v for k, v in label_info["id2label"].items()}

3. 의료용어 추출:
   def extract_medical_entities(text):
       tokens = text.split()
       inputs = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
       outputs = model(**inputs)
       predictions = outputs.logits.argmax(dim=-1)

       results = []
       for i, pred_id in enumerate(predictions[0]):
           if i < len(tokens):
               label = id2label[pred_id.item()]
               if label != 'O':
                   results.append((tokens[i], label))
       return results

4. 사용 예시:
   text = "환자는 고혈압 진단을 받고 인슐린 치료를 시작했습니다"
   entities = extract_medical_entities(text)
   print(entities)  # [('고혈압', 'Disease-B'), ('인슐린', 'Drug-B'), ...]

🏷️ 라벨 체계:
- O: 일반 단어
- Disease-B/I: 질병명 시작/내부
- Drug-B/I: 약물명 시작/내부
- Procedure-B/I: 시술/검사명 시작/내부
- 기타 의료 관련 라벨들...

📊 성능:
- 학습 데이터: KBMC 데이터셋 (약 5,000개 문장)
- 모델: KoELECTRA-base-v3-discriminator
- 정확도: 약 90%+ (검증 데이터 기준)

⚠️ 주의사항:
- 의료 전문 용도로 사용 시 반드시 전문가 검토 필요
- 이 모델은 연구/개발 목적으로 학습됨
- 실제 의료 현장 적용 시 충분한 검증 필요

📧 문의: AI 개발팀
📅 생성일: {현재 날짜}
"""

with open("NER_모델_사용법.txt", "w", encoding="utf-8") as f:
    f.write(usage_guide)

print("✅ 사용법 가이드 생성 완료: NER_모델_사용법.txt")

print("\n" + "="*50)

# ==========================================
# 4단계: 다운로드 실행
# ==========================================

print("⬇️ === 파일 다운로드 ===")

if zip_file and os.path.exists(zip_file):
    try:
        print(f"🔄 {zip_file} 다운로드 중...")
        files.download(zip_file)
        print("✅ 모델 ZIP 파일 다운로드 완료!")
    except Exception as e:
        print(f"❌ ZIP 다운로드 실패: {e}")

try:
    print("🔄 사용법 가이드 다운로드 중...")
    files.download("NER_모델_사용법.txt")
    print("✅ 사용법 가이드 다운로드 완료!")
except Exception as e:
    print(f"❌ 가이드 다운로드 실패: {e}")

print("\n" + "="*50)

# ==========================================
# 5단계: 추가 백업 정보
# ==========================================

print("📋 === 백업 정보 요약 ===")

backup_info = {
    "모델_경로": "./kbmc-ner-final/",
    "압축_파일": zip_file,
    "라벨_수": len(label2id) if 'label2id' in globals() else "알 수 없음",
    "학습_데이터": "KBMC 데이터셋",
    "모델_타입": "KoELECTRA + NER",
    "용도": "한국어 의료 개체명 인식"
}

print("백업 완료 정보:")
for key, value in backup_info.items():
    print(f"  {key}: {value}")

print("\n🎉 === 모든 백업 완료! ===")
print("✅ 다운로드된 파일들:")
print("   1. kbmc-ner-model.zip (학습된 모델)")
print("   2. NER_모델_사용법.txt (사용 가이드)")
print("\n💡 이 파일들을 안전한 곳에 보관하세요!")
print("🔄 나중에 이 모델로 심평원 데이터에서 의료용어를 추출할 수 있습니다!")

📦 === 학습된 모델 다운로드 준비 ===
📁 kbmc-ner-final 폴더 내용:
  checkpoint-924 (0.0 MB)
  label_mapping.json (0.0 MB)
  tokenizer_config.json (0.0 MB)
  special_tokens_map.json (0.0 MB)
  vocab.txt (0.3 MB)
  config.json (0.0 MB)
  tokenizer.json (0.8 MB)
  model.safetensors (428.6 MB)

🗜️ === 모델 압축 중 ===
  추가: kbmc-ner-final/label_mapping.json
  추가: kbmc-ner-final/tokenizer_config.json
  추가: kbmc-ner-final/special_tokens_map.json
  추가: kbmc-ner-final/vocab.txt
  추가: kbmc-ner-final/config.json
  추가: kbmc-ner-final/tokenizer.json
  추가: kbmc-ner-final/model.safetensors
  추가: kbmc-ner-final/checkpoint-924/tokenizer_config.json
  추가: kbmc-ner-final/checkpoint-924/rng_state.pth
  추가: kbmc-ner-final/checkpoint-924/special_tokens_map.json
  추가: kbmc-ner-final/checkpoint-924/vocab.txt
  추가: kbmc-ner-final/checkpoint-924/training_args.bin
  추가: kbmc-ner-final/checkpoint-924/config.json
  추가: kbmc-ner-final/checkpoint-924/optimizer.pt
  추가: kbmc-ner-final/checkpoint-924/tokenizer.json
  추가: kbmc-ner-final/ch

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 모델 ZIP 파일 다운로드 완료!
🔄 사용법 가이드 다운로드 중...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ 사용법 가이드 다운로드 완료!

📋 === 백업 정보 요약 ===
백업 완료 정보:
  모델_경로: ./kbmc-ner-final/
  압축_파일: kbmc-ner-model.zip
  라벨_수: 7
  학습_데이터: KBMC 데이터셋
  모델_타입: KoELECTRA + NER
  용도: 한국어 의료 개체명 인식

🎉 === 모든 백업 완료! ===
✅ 다운로드된 파일들:
   1. kbmc-ner-model.zip (학습된 모델)
   2. NER_모델_사용법.txt (사용 가이드)

💡 이 파일들을 안전한 곳에 보관하세요!
🔄 나중에 이 모델로 심평원 데이터에서 의료용어를 추출할 수 있습니다!


In [ ]:
# 📁 NER 폴더들을 구글 드라이브로 이동

print("📁 === NER 폴더들을 구글 드라이브로 이동 ===")

import os
import shutil
from pathlib import Path

# ==========================================
# 1단계: 구글 드라이브 마운트 확인
# ==========================================

# 구글 드라이브가 마운트되어 있는지 확인
if not os.path.exists('/content/drive'):
    print("🔗 구글 드라이브 마운트 중...")
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("✅ 구글 드라이브 이미 마운트됨")

# 목표 경로 설정
target_base_path = "/content/drive/MyDrive/CGINSIDE"
ner_target_path = "/content/drive/MyDrive/CGINSIDE/NER"

print(f"목표 경로: {ner_target_path}")

print("\n" + "="*50)

# ==========================================
# 2단계: 목표 디렉토리 생성
# ==========================================

print("📂 === 목표 디렉토리 생성 ===")

# CGINSIDE 폴더 생성 (없으면)
if not os.path.exists(target_base_path):
    os.makedirs(target_base_path)
    print(f"✅ 생성: {target_base_path}")
else:
    print(f"✅ 이미 존재: {target_base_path}")

# NER 폴더 생성 (없으면)
if not os.path.exists(ner_target_path):
    os.makedirs(ner_target_path)
    print(f"✅ 생성: {ner_target_path}")
else:
    print(f"✅ 이미 존재: {ner_target_path}")

print("\n" + "="*50)

# ==========================================
# 3단계: 폴더 이동 함수
# ==========================================

def move_folder_to_drive(source_folder, target_parent, folder_name):
    """폴더를 구글 드라이브로 이동"""

    source_path = f"./{source_folder}"
    target_path = f"{target_parent}/{folder_name}"

    if not os.path.exists(source_path):
        print(f"❌ 소스 폴더 없음: {source_path}")
        return False

    try:
        # 대상 폴더가 이미 있으면 삭제
        if os.path.exists(target_path):
            print(f"🗑️ 기존 폴더 삭제: {target_path}")
            shutil.rmtree(target_path)

        # 폴더 복사
        print(f"📁 복사 중: {source_path} → {target_path}")
        shutil.copytree(source_path, target_path)

        # 복사 완료 후 원본 삭제
        print(f"🗑️ 원본 삭제: {source_path}")
        shutil.rmtree(source_path)

        # 파일 개수 확인
        file_count = sum([len(files) for r, d, files in os.walk(target_path)])
        total_size = sum([os.path.getsize(os.path.join(r, f)) for r, d, files in os.walk(target_path) for f in files])
        size_mb = total_size / (1024 * 1024)

        print(f"✅ 이동 완료: {folder_name}")
        print(f"   파일 수: {file_count}개")
        print(f"   크기: {size_mb:.1f} MB")

        return True

    except Exception as e:
        print(f"❌ 이동 실패 ({folder_name}): {e}")
        return False

# ==========================================
# 4단계: 폴더들 이동 실행
# ==========================================

print("🚀 === 폴더 이동 실행 ===")

folders_to_move = [
    ("kbmc-ner-final", "kbmc-ner-final"),
    ("kbmc-ner-koelectra", "kbmc-ner-koelectra")
]

success_count = 0
total_count = len(folders_to_move)

for source_folder, target_name in folders_to_move:
    print(f"\n📦 {source_folder} 이동 중...")

    if move_folder_to_drive(source_folder, ner_target_path, target_name):
        success_count += 1

    print("-" * 30)

print(f"\n📊 이동 결과: {success_count}/{total_count} 성공")

print("\n" + "="*50)

# ==========================================
# 5단계: 이동 결과 확인
# ==========================================

print("🔍 === 이동 결과 확인 ===")

if os.path.exists(ner_target_path):
    print(f"📁 {ner_target_path} 내용:")

    for item in os.listdir(ner_target_path):
        item_path = os.path.join(ner_target_path, item)

        if os.path.isdir(item_path):
            # 폴더인 경우 내부 파일 수와 크기 계산
            file_count = sum([len(files) for r, d, files in os.walk(item_path)])
            total_size = sum([os.path.getsize(os.path.join(r, f)) for r, d, files in os.walk(item_path) for f in files])
            size_mb = total_size / (1024 * 1024)

            print(f"  📂 {item}/ ({file_count}개 파일, {size_mb:.1f} MB)")

            # 주요 파일들 미리보기
            main_files = []
            for root, dirs, files in os.walk(item_path):
                for file in files[:5]:  # 처음 5개만
                    main_files.append(file)

            if main_files:
                print(f"     주요 파일: {', '.join(main_files[:3])}{'...' if len(main_files) > 3 else ''}")

        else:
            # 파일인 경우
            size_mb = os.path.getsize(item_path) / (1024 * 1024)
            print(f"  📄 {item} ({size_mb:.1f} MB)")

else:
    print(f"❌ 대상 경로가 존재하지 않습니다: {ner_target_path}")

print("\n" + "="*50)

# ==========================================
# 6단계: 추가 백업 파일 생성
# ==========================================

print("📝 === 백업 정보 파일 생성 ===")

backup_info = f"""
🤖 NER 모델 백업 정보

📅 백업 일시: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S') if 'pd' in globals() else '2024년'}
📍 백업 위치: {ner_target_path}

📁 백업된 폴더:
1. kbmc-ner-final/ - 최종 완성된 NER 모델
   - config.json: 모델 설정
   - pytorch_model.bin: 학습된 가중치
   - tokenizer 파일들: 토크나이저
   - label_mapping.json: 라벨 매핑

2. kbmc-ner-koelectra/ - 학습 중간 체크포인트
   - 학습 로그 및 중간 저장 파일들

🔧 모델 정보:
- 기반 모델: KoELECTRA-base-v3-discriminator
- 학습 데이터: KBMC 한국어 의료 NER 데이터셋
- 라벨 수: {len(label2id) if 'label2id' in globals() else '알 수 없음'}개
- 용도: 한국어 의료 개체명 인식

🚀 사용법:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_path = "{ner_target_path}/kbmc-ner-final"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

💡 활용 예시:
- 심평원 데이터에서 의료용어 추출
- 의료 문서 자동 태깅
- 의료 정보 추출 시스템

⚠️ 주의사항:
- 의료 전문 용도 사용 시 전문가 검증 필요
- 연구/개발 목적으로 학습된 모델
"""

backup_info_path = f"{ner_target_path}/NER_백업_정보.txt"

try:
    with open(backup_info_path, "w", encoding="utf-8") as f:
        f.write(backup_info)
    print(f"✅ 백업 정보 파일 생성: {backup_info_path}")
except Exception as e:
    print(f"❌ 백업 정보 파일 생성 실패: {e}")

print("\n" + "="*50)

print("🎉 === 모든 작업 완료! ===")
print(f"✅ NER 모델 폴더들이 구글 드라이브로 이동되었습니다!")
print(f"📍 위치: {ner_target_path}")
print(f"🔗 구글 드라이브에서 확인하세요: MyDrive/CGINSIDE/NER/")
print("\n💡 이제 세션이 종료되어도 모델이 안전하게 보관됩니다!")
print("🔄 나중에 이 경로에서 모델을 로드하여 사용할 수 있습니다!")

# 현재 작업 디렉토리 정리
print(f"\n🧹 현재 디렉토리 정리...")
current_files = [f for f in os.listdir('.') if f.startswith('kbmc-ner')]
if current_files:
    print(f"남은 파일: {current_files} (이미 이동됨)")
else:
    print("✅ 모든 NER 파일이 구글 드라이브로 이동됨")

📁 === NER 폴더들을 구글 드라이브로 이동 ===
✅ 구글 드라이브 이미 마운트됨
목표 경로: /content/drive/MyDrive/CGINSIDE/NER

📂 === 목표 디렉토리 생성 ===
✅ 이미 존재: /content/drive/MyDrive/CGINSIDE
✅ 생성: /content/drive/MyDrive/CGINSIDE/NER

🚀 === 폴더 이동 실행 ===

📦 kbmc-ner-final 이동 중...
📁 복사 중: ./kbmc-ner-final → /content/drive/MyDrive/CGINSIDE/NER/kbmc-ner-final
🗑️ 원본 삭제: ./kbmc-ner-final
✅ 이동 완료: kbmc-ner-final
   파일 수: 19개
   크기: 1716.4 MB
------------------------------

📦 kbmc-ner-koelectra 이동 중...
📁 복사 중: ./kbmc-ner-koelectra → /content/drive/MyDrive/CGINSIDE/NER/kbmc-ner-koelectra
🗑️ 원본 삭제: ./kbmc-ner-koelectra
✅ 이동 완료: kbmc-ner-koelectra
   파일 수: 1개
   크기: 0.0 MB
------------------------------

📊 이동 결과: 2/2 성공

🔍 === 이동 결과 확인 ===
📁 /content/drive/MyDrive/CGINSIDE/NER 내용:
  📂 kbmc-ner-final/ (19개 파일, 1716.4 MB)
     주요 파일: label_mapping.json, tokenizer_config.json, special_tokens_map.json...
  📂 kbmc-ner-koelectra/ (1개 파일, 0.0 MB)
     주요 파일: events.out.tfevents.1755662439.60b8ea0e9e47.756.0

📝 === 백업 정보 파일 생성 ===
✅ 백업 정보 파

In [ ]:
# 🏥 학습된 NER 모델로 심평원 데이터 의료용어 추출

print("🤖 === NER 모델 기반 의료용어 추출 시작 ===")

import pandas as pd
import torch
import json
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForTokenClassification
import time

# ==========================================
# 1단계: 학습된 NER 모델 로드
# ==========================================

print("📥 === NER 모델 로드 ===")

# 구글 드라이브에서 모델 로드
model_path = "/content/drive/MyDrive/CGINSIDE/NER/kbmc-ner-final"

try:
    # 모델과 토크나이저 로드
    print("🔄 모델 로드 중...")
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # GPU로 이동
    if torch.cuda.is_available():
        model = model.cuda()
        print("✅ 모델을 GPU로 이동 완료")

    # 라벨 매핑 로드
    with open(f"{model_path}/label_mapping.json", "r", encoding="utf-8") as f:
        label_info = json.load(f)

    id2label = {int(k): v for k, v in label_info["id2label"].items()}
    label2id = label_info["label2id"]
    labels = label_info["labels"]

    print(f"✅ NER 모델 로드 완료!")
    print(f"📊 라벨 수: {len(labels)}개")
    print(f"🏷️ 주요 라벨: {labels[:10]}...")

except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("💡 구글 드라이브가 마운트되어 있는지 확인하세요!")
    raise e

print("\n" + "="*50)

# ==========================================
# 2단계: 의료용어 추출 함수 정의
# ==========================================

print("⚙️ === 의료용어 추출 함수 정의 ===")

class MedicalNERExtractor:
    def __init__(self, model, tokenizer, id2label, confidence_threshold=0.7):
        self.model = model
        self.tokenizer = tokenizer
        self.id2label = id2label
        self.confidence_threshold = confidence_threshold
        self.model.eval()

    def extract_entities_from_text(self, text, max_length=512):
        """텍스트에서 의료 개체명 추출"""
        if not text or len(text.strip()) < 3:
            return []

        # 텍스트를 적절한 길이로 분할
        words = text.split()
        if len(words) > max_length // 2:  # 안전한 길이로 자르기
            words = words[:max_length // 2]

        try:
            # 토크나이징
            inputs = self.tokenizer(
                words,
                is_split_into_words=True,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=max_length
            )

            # GPU로 이동
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            # 예측
            with torch.no_grad():
                outputs = self.model(**inputs)
                probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predictions = probabilities.argmax(dim=-1)

            # 결과 추출
            entities = []
            current_entity = ""
            current_label = None
            current_confidence = 0.0

            # 토큰별 처리
            tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            predictions_cpu = predictions[0].cpu().numpy()
            probabilities_cpu = probabilities[0].cpu().numpy()

            word_idx = 0
            for i, (token, pred_id) in enumerate(zip(tokens, predictions_cpu)):

                # 특수 토큰 건너뛰기
                if token in ['[CLS]', '[SEP]', '[PAD]']:
                    continue

                confidence = probabilities_cpu[i][pred_id]
                label = self.id2label[pred_id]

                # 신뢰도 필터링
                if confidence < self.confidence_threshold:
                    # 현재 개체 저장
                    if current_entity and current_label != 'O':
                        entities.append({
                            'text': current_entity.strip(),
                            'label': current_label,
                            'confidence': current_confidence
                        })
                    current_entity = ""
                    current_label = None
                    continue

                # 서브워드 토큰 처리
                if token.startswith('##'):
                    if current_entity:
                        current_entity += token[2:]  # ## 제거 후 추가
                    continue

                # B- 태그 (새로운 개체 시작)
                if label.endswith('-B'):
                    # 이전 개체 저장
                    if current_entity and current_label != 'O':
                        entities.append({
                            'text': current_entity.strip(),
                            'label': current_label,
                            'confidence': current_confidence
                        })

                    # 새로운 개체 시작
                    current_entity = token
                    current_label = label[:-2]  # -B 제거
                    current_confidence = confidence

                # I- 태그 (개체 연속)
                elif label.endswith('-I') and current_label == label[:-2]:
                    current_entity += " " + token
                    current_confidence = (current_confidence + confidence) / 2  # 평균 신뢰도

                # O 태그 또는 다른 개체
                else:
                    # 이전 개체 저장
                    if current_entity and current_label != 'O':
                        entities.append({
                            'text': current_entity.strip(),
                            'label': current_label,
                            'confidence': current_confidence
                        })

                    current_entity = ""
                    current_label = None

            # 마지막 개체 저장
            if current_entity and current_label != 'O':
                entities.append({
                    'text': current_entity.strip(),
                    'label': current_label,
                    'confidence': current_confidence
                })

            # 후처리: 중복 제거 및 정리
            cleaned_entities = []
            for entity in entities:
                text = entity['text'].strip()
                # 너무 짧거나 빈 텍스트 제거
                if len(text) >= 2 and not re.match(r'^[^\w가-힣]+$', text):
                    entity['text'] = text
                    cleaned_entities.append(entity)

            return cleaned_entities

        except Exception as e:
            print(f"❌ 텍스트 처리 오류: {e}")
            return []

    def extract_from_dataframe(self, df, text_columns=['title', 'content'], batch_size=100):
        """데이터프레임에서 의료용어 대량 추출"""

        all_entities = Counter()
        entity_details = []

        total_rows = len(df)
        print(f"📊 총 {total_rows:,}개 문서 처리 시작...")

        start_time = time.time()

        for idx, row in df.iterrows():

            # 진행률 표시
            if idx % 500 == 0:
                elapsed = time.time() - start_time
                progress = idx / total_rows * 100
                eta = elapsed * (total_rows - idx) / idx if idx > 0 else 0

                print(f"📈 진행률: {idx:,}/{total_rows:,} ({progress:.1f}%) | "
                      f"경과: {elapsed/60:.1f}분 | 예상 잔여: {eta/60:.1f}분")

            # 각 텍스트 컬럼에서 추출
            for col in text_columns:
                text = str(row.get(col, ''))

                if text and text != 'nan' and len(text.strip()) > 5:
                    try:
                        entities = self.extract_entities_from_text(text)

                        # 가중치 적용 (제목이 더 중요)
                        weight = 3 if col == 'title' else 1

                        for entity in entities:
                            term = entity['text']
                            label = entity['label']
                            confidence = entity['confidence']

                            # 신뢰도가 높은 것만 저장
                            if confidence >= self.confidence_threshold:
                                all_entities[term] += weight

                                entity_details.append({
                                    'term': term,
                                    'label': label,
                                    'confidence': confidence,
                                    'source': col,
                                    'doc_index': idx,
                                    'weight': weight
                                })

                    except Exception as e:
                        print(f"❌ 문서 {idx} 처리 오류: {e}")
                        continue

        total_time = time.time() - start_time
        print(f"\n✅ 추출 완료! 총 소요시간: {total_time/60:.1f}분")
        print(f"📊 추출된 고유 의료용어: {len(all_entities):,}개")

        return dict(all_entities), entity_details

# 추출기 생성
extractor = MedicalNERExtractor(model, tokenizer, id2label, confidence_threshold=0.6)
print("✅ 의료용어 추출기 생성 완료")

print("\n" + "="*50)

# ==========================================
# 3단계: 심평원 데이터 로드 및 추출
# ==========================================

print("📂 === 심평원 데이터 로드 ===")

# 먼저 파일 확인
import os
print("📁 현재 디렉토리 파일들:")
for file in os.listdir('.'):
    if file.endswith('.xlsx'):
        size = os.path.getsize(file) / (1024*1024)
        print(f"  📊 {file} ({size:.1f} MB)")

try:
    # 심평원 데이터 로드 (업로드된 파일)
    df = pd.read_excel('/content/drive/MyDrive/CGINSIDE/보험심사/hira_datas.xlsx')
    print(f"\n✅ 심평원 데이터 로드 완료: {len(df):,}개 문서")
    print(f"📋 컬럼: {df.columns.tolist()}")

    # 데이터 구조 확인
    print(f"📊 데이터 형태: {df.shape}")
    print(f"📝 컬럼별 샘플:")
    for col in df.columns:
        sample = str(df[col].iloc[0])[:50] if len(df) > 0 else "없음"
        print(f"  {col}: {sample}...")

except Exception as e:
    print(f"❌ 데이터 로드 실패: {e}")
    print("💡 파일 읽기 오류 - 다른 방법으로 시도합니다...")

    # 대안: 파일을 다시 읽어보기
    try:
        df = pd.read_excel('/content/drive/MyDrive/CGINSIDE/보험심사/hira_datas.xlsx', engine='openpyxl')
        print(f"✅ 대안 방법으로 로드 성공: {len(df):,}개 문서")
    except Exception as e2:
        print(f"❌ 대안 방법도 실패: {e2}")
        raise e2

print("\n" + "="*50)

# ==========================================
# 4단계: 의료용어 추출 실행
# ==========================================

print("🚀 === NER 기반 의료용어 추출 실행 ===")
print("⏰ 예상 소요시간: 20-40분 (데이터 크기에 따라)")
print("🎯 신뢰도 임계값: 0.6 (60% 이상)")

# 추출 실행
medical_terms, entity_details = extractor.extract_from_dataframe(
    df,
    text_columns=['title', 'content']
)

print(f"\n🎊 추출 결과:")
print(f"📊 총 고유 의료용어: {len(medical_terms):,}개")
print(f"📝 총 추출 인스턴스: {len(entity_details):,}개")

print("\n" + "="*50)

# ==========================================
# 5단계: 결과 분석 및 저장
# ==========================================

print("📈 === 결과 분석 ===")

# 빈도별 정렬
sorted_terms = sorted(medical_terms.items(), key=lambda x: x[1], reverse=True)

# 라벨별 분포
label_counts = Counter([detail['label'] for detail in entity_details])
print(f"\n🏷️ 라벨별 분포:")
for label, count in label_counts.most_common():
    percentage = count / len(entity_details) * 100
    print(f"  {label:15s}: {count:,}개 ({percentage:.1f}%)")

# 신뢰도 분포
confidences = [detail['confidence'] for detail in entity_details]
avg_confidence = sum(confidences) / len(confidences)
print(f"\n🎯 평균 신뢰도: {avg_confidence:.3f}")

# 빈도 분포
frequencies = list(medical_terms.values())
print(f"\n📊 빈도 분포:")
print(f"  최고 빈도: {max(frequencies):,}회")
print(f"  평균 빈도: {sum(frequencies)/len(frequencies):.1f}회")

# 고빈도 용어들
print(f"\n🔝 상위 50개 의료용어:")
for rank, (term, freq) in enumerate(sorted_terms[:50], 1):
    print(f"{rank:2d}. {term:20s} ({freq:3d}회)")

print("\n" + "="*50)

# ==========================================
# 6단계: 결과 저장
# ==========================================

print("💾 === 결과 저장 ===")

# CSV 저장
csv_filename = "ner_medical_terms_final.csv"

try:
    with open(csv_filename, 'w', encoding='utf-8') as f:
        f.write("rank,term,frequency,avg_confidence,main_label\n")

        for rank, (term, freq) in enumerate(sorted_terms, 1):
            # 해당 용어의 상세 정보 계산
            term_details = [d for d in entity_details if d['term'] == term]

            if term_details:
                avg_conf = sum(d['confidence'] for d in term_details) / len(term_details)
                main_label = Counter(d['label'] for d in term_details).most_common(1)[0][0]
            else:
                avg_conf = 0.0
                main_label = "Unknown"

            f.write(f"{rank},{term},{freq},{avg_conf:.3f},{main_label}\n")

    print(f"✅ CSV 저장 완료: {csv_filename}")

    # 구글 드라이브에도 저장
    drive_path = "/content/drive/MyDrive/CGINSIDE/NER/ner_medical_terms_final.csv"
    import shutil
    shutil.copy(csv_filename, drive_path)
    print(f"✅ 구글 드라이브 저장: {drive_path}")

except Exception as e:
    print(f"❌ 저장 실패: {e}")

# 요약 리포트 생성
report = f"""
🏥 NER 기반 의료용어 추출 리포트

📅 추출 일시: {pd.Timestamp.now()}
📊 분석 데이터: {len(df):,}개 심평원 문서
🤖 사용 모델: KBMC-NER (KoELECTRA 기반)

📈 추출 결과:
- 총 고유 의료용어: {len(medical_terms):,}개
- 총 추출 인스턴스: {len(entity_details):,}개
- 평균 신뢰도: {avg_confidence:.3f}
- 신뢰도 임계값: 0.6

🏷️ 주요 라벨:
{chr(10).join([f"- {label}: {count:,}개" for label, count in label_counts.most_common(5)])}

🔝 최고 빈도 용어:
{chr(10).join([f"- {term}: {freq}회" for term, freq in sorted_terms[:10]])}

💡 활용 방안:
- 의료 문서 자동 태깅
- 의료 용어 사전 구축
- 진료 기록 분석
- 의료 정보 추출 시스템
"""

with open("ner_extraction_report.txt", "w", encoding="utf-8") as f:
    f.write(report)

print(f"✅ 리포트 저장: ner_extraction_report.txt")

print("\n🎉 === NER 기반 의료용어 추출 완료! ===")
print(f"✅ 총 {len(medical_terms):,}개 의료용어 추출 성공")
print(f"📁 결과 파일: {csv_filename}")
print(f"📋 리포트: ner_extraction_report.txt")
print(f"☁️ 구글 드라이브에도 백업됨")

🤖 === NER 모델 기반 의료용어 추출 시작 ===
📥 === NER 모델 로드 ===
🔄 모델 로드 중...
✅ 모델을 GPU로 이동 완료
✅ NER 모델 로드 완료!
📊 라벨 수: 7개
🏷️ 주요 라벨: ['Body-B', 'Body-I', 'Disease-B', 'Disease-I', 'O', 'Treatment-B', 'Treatment-I']...

⚙️ === 의료용어 추출 함수 정의 ===
✅ 의료용어 추출기 생성 완료

📂 === 심평원 데이터 로드 ===
📁 현재 디렉토리 파일들:

✅ 심평원 데이터 로드 완료: 8,413개 문서
📋 컬럼: ['published_date', 'title', 'category', 'detail_category', 'relevant', 'content', 'files_local', 'file_urls']
📊 데이터 형태: (8413, 8)
📝 컬럼별 샘플:
  published_date: 2025-07-01...
  title: 슬관절강내 주입용 폴리뉴클레오티드나트륨의 급여기준...
  category: 고시...
  detail_category: 고시...
  relevant: 고시 제2024-184호...
  content: ★ 2025.2.26.~
[집행정지 안내] 요양급여의 적용기준 및 방법에 관한 세부사항(고...
  files_local: ['selenium\\고시\\2025-07-01_고시_슬관절강내 주입용 폴리뉴클레오티드나트...
  file_urls: [{'url': 'https://www.hira.or.kr/download.do?src=%...

🚀 === NER 기반 의료용어 추출 실행 ===
⏰ 예상 소요시간: 20-40분 (데이터 크기에 따라)
🎯 신뢰도 임계값: 0.6 (60% 이상)
📊 총 8,413개 문서 처리 시작...
📈 진행률: 0/8,413 (0.0%) | 경과: 0.0분 | 예상 잔여: 0.0분
📈 진행률: 500/8,413 (5.9%) | 경과: 0.3분 | 예상 잔여: 

In [ ]:
# 🔧 고품질 의료용어 추출 (불용어 제거 + 후처리)

print("🧹 === 고품질 의료용어 추출 시작 ===")

import pandas as pd
import torch
import json
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForTokenClassification
import time

# ==========================================
# 1단계: 의료 불용어 및 필터링 규칙 정의
# ==========================================

print("📋 === 의료용어 필터링 규칙 정의 ===")

# 의료 불용어 (너무 일반적이거나 의미없는 용어들)
MEDICAL_STOPWORDS = {
    # 너무 일반적인 의료 용어
    '수술', '수술을', '수술의', '수술에서', '수술시', '수술후',
    '치료', '치료를', '치료의', '치료에', '치료법', '치료제',
    '검사', '검사를', '검사의', '검사에서', '검사시', '검사후',
    '진료', '진료를', '진료의', '진료에서', '진료시', '진료후',
    '처치', '처치를', '처치의', '처치에서', '처치시', '처치후',
    '시술', '시술을', '시술의', '시술에서', '시술시', '시술후',

    # 제형 관련 일반 용어
    '주사제', '경구제', '정제', '캡슐', '시럽', '연고', '겔',
    '외용제', '내복약', '주사약', '정맥주사', '근육주사',

    # 너무 일반적인 해부학 용어
    '피부', '피부의', '뼈', '뼈의', '근육', '근육의',
    '혈관', '혈관의', '신경', '신경의', '조직', '조직의',

    # 행정/절차 용어
    '요양급', '선별급여', '급여', '급여의', '비급여',
    '외래', '외래의', '입원', '입원의', '퇴원', '퇴원의',
    '간호', '간호의', '간병', '간병의',

    # 상태 표현
    '상세불명의', '기타', '기타의', '미상', '미상의',
    '증상', '증상의', '소견', '소견의', '상태', '상태의',

    # 일반 단어
    '환자', '환자의', '질환', '질환의', '질병', '질병의',
    '통증', '통증의', '출혈', '출혈의', '외상', '외상의',
    '종양', '종양의', '암', '암의', '의한', '등의', '관련',
}

# 한국어 조사 패턴
KOREAN_PARTICLES = r'[을를이가에서로으로와과의은는]$|[이가]$|에서$|으로$|와의$|치아에$|유방의$'

# 보존할 고품질 의료용어 패턴
HIGH_QUALITY_PATTERNS = {
    # 구체적인 질병명
    'specific_diseases': [
        r'.*경색.*', r'.*증후군.*', r'.*염.*', r'.*종.*', r'.*암.*',
        r'.*당뇨.*', r'.*고혈압.*', r'.*뇌졸중.*', r'.*폐렴.*',
        r'.*간염.*', r'.*신부전.*', r'.*부정맥.*'
    ],

    # 구체적인 시술/수술명
    'specific_procedures': [
        r'.*이식.*', r'.*치환술.*', r'.*고정술.*', r'.*삽입술.*',
        r'.*절제술.*', r'.*성형술.*', r'.*복원술.*', r'.*재건술.*',
        r'.*투석.*', r'.*요법.*', r'.*면역.*검사.*'
    ],

    # 의료기기/장비
    'medical_devices': [
        r'MRI', r'CT', r'VAD', r'ICD', r'.*내시경.*',
        r'.*카테터.*', r'.*스텐트.*', r'.*임플란트.*'
    ],

    # 구체적 해부학 부위
    'specific_anatomy': [
        r'.*추.*', r'.*관절.*', r'.*판.*장애.*',
        r'구순구개열.*', r'.*심방.*', r'.*심실.*'
    ]
}

def is_high_quality_medical_term(term):
    """고품질 의료용어인지 판단"""

    # 너무 짧거나 긴 용어 제외
    if len(term) < 3 or len(term) > 30:
        return False

    # 불용어 체크
    if term in MEDICAL_STOPWORDS:
        return False

    # 조사 붙은 용어 제외
    if re.search(KOREAN_PARTICLES, term):
        return False

    # 숫자만 있는 용어 제외
    if re.match(r'^[\d\s\-\.]+$', term):
        return False

    # 영문자만 있고 너무 짧은 용어 제외
    if re.match(r'^[A-Za-z\s]{1,3}$', term):
        return False

    # 특수문자가 너무 많은 용어 제외
    if len(re.findall(r'[^\w가-힣\s]', term)) > len(term) * 0.3:
        return False

    # 고품질 패턴 중 하나라도 매치되면 보존
    for category, patterns in HIGH_QUALITY_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, term, re.IGNORECASE):
                return True

    # 추가 의료용어 휴리스틱
    medical_suffixes = ['증', '병', '염', '종', '암', '술', '법', '요법', '검사', '촬영']
    if any(term.endswith(suffix) for suffix in medical_suffixes):
        return True

    # 의료 접두사
    medical_prefixes = ['급성', '만성', '외상성', '감염성', '자가면역']
    if any(term.startswith(prefix) for prefix in medical_prefixes):
        return True

    return False

print("✅ 필터링 규칙 정의 완료")
print(f"📝 불용어 수: {len(MEDICAL_STOPWORDS)}개")

print("\n" + "="*50)

# ==========================================
# 2단계: 개선된 NER 추출기
# ==========================================

print("🤖 === 개선된 NER 추출기 ===")

class RefinedMedicalNERExtractor:
    def __init__(self, model, tokenizer, id2label, confidence_threshold=0.75):
        self.model = model
        self.tokenizer = tokenizer
        self.id2label = id2label
        self.confidence_threshold = confidence_threshold
        self.model.eval()

    def clean_extracted_term(self, term):
        """추출된 용어 정리"""

        # 앞뒤 공백 제거
        term = term.strip()

        # 특수문자 정리
        term = re.sub(r'^[^\w가-힣]+|[^\w가-힣]+$', '', term)

        # 연속된 공백 정리
        term = re.sub(r'\s+', ' ', term)

        # 조사 제거
        term = re.sub(KOREAN_PARTICLES, '', term)

        return term.strip()

    def extract_entities_from_text(self, text, max_length=256):
        """텍스트에서 고품질 의료 개체명 추출"""

        if not text or len(text.strip()) < 3:
            return []

        # 텍스트 전처리
        text = re.sub(r'\s+', ' ', text.strip())
        words = text.split()

        # 길이 제한
        if len(words) > max_length // 3:
            words = words[:max_length // 3]

        try:
            # 토크나이징
            inputs = self.tokenizer(
                words,
                is_split_into_words=True,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=max_length
            )

            # GPU로 이동
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            # 예측
            with torch.no_grad():
                outputs = self.model(**inputs)
                probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predictions = probabilities.argmax(dim=-1)

            # 결과 추출 (개선된 로직)
            entities = []
            current_entity = ""
            current_label = None
            current_confidences = []

            tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            predictions_cpu = predictions[0].cpu().numpy()
            probabilities_cpu = probabilities[0].cpu().numpy()

            for i, (token, pred_id) in enumerate(zip(tokens, predictions_cpu)):

                if token in ['[CLS]', '[SEP]', '[PAD]']:
                    continue

                confidence = probabilities_cpu[i][pred_id]
                label = self.id2label[pred_id]

                # 높은 신뢰도만 처리
                if confidence < self.confidence_threshold:
                    # 현재 개체 저장
                    if current_entity and current_label != 'O':
                        avg_conf = sum(current_confidences) / len(current_confidences)
                        cleaned_term = self.clean_extracted_term(current_entity)

                        if cleaned_term and is_high_quality_medical_term(cleaned_term):
                            entities.append({
                                'text': cleaned_term,
                                'label': current_label,
                                'confidence': avg_conf
                            })

                    current_entity = ""
                    current_label = None
                    current_confidences = []
                    continue

                # 서브워드 처리
                if token.startswith('##'):
                    if current_entity:
                        current_entity += token[2:]
                        current_confidences.append(confidence)
                    continue

                # B- 태그 (새로운 개체)
                if label.endswith('-B'):
                    # 이전 개체 저장
                    if current_entity and current_label != 'O':
                        avg_conf = sum(current_confidences) / len(current_confidences)
                        cleaned_term = self.clean_extracted_term(current_entity)

                        if cleaned_term and is_high_quality_medical_term(cleaned_term):
                            entities.append({
                                'text': cleaned_term,
                                'label': current_label,
                                'confidence': avg_conf
                            })

                    # 새로운 개체 시작
                    current_entity = token
                    current_label = label[:-2]
                    current_confidences = [confidence]

                # I- 태그 (개체 연속)
                elif label.endswith('-I') and current_label == label[:-2]:
                    current_entity += " " + token
                    current_confidences.append(confidence)

                # O 태그 또는 다른 개체
                else:
                    # 이전 개체 저장
                    if current_entity and current_label != 'O':
                        avg_conf = sum(current_confidences) / len(current_confidences)
                        cleaned_term = self.clean_extracted_term(current_entity)

                        if cleaned_term and is_high_quality_medical_term(cleaned_term):
                            entities.append({
                                'text': cleaned_term,
                                'label': current_label,
                                'confidence': avg_conf
                            })

                    current_entity = ""
                    current_label = None
                    current_confidences = []

            # 마지막 개체 저장
            if current_entity and current_label != 'O':
                avg_conf = sum(current_confidences) / len(current_confidences)
                cleaned_term = self.clean_extracted_term(current_entity)

                if cleaned_term and is_high_quality_medical_term(cleaned_term):
                    entities.append({
                        'text': cleaned_term,
                        'label': current_label,
                        'confidence': avg_conf
                    })

            return entities

        except Exception as e:
            print(f"❌ 텍스트 처리 오류: {e}")
            return []

    def extract_from_dataframe(self, df, text_columns=['title', 'content']):
        """데이터프레임에서 고품질 의료용어 대량 추출"""

        all_entities = Counter()
        entity_details = []

        total_rows = len(df)
        print(f"📊 총 {total_rows:,}개 문서 처리 시작...")

        start_time = time.time()

        for idx, row in df.iterrows():

            # 진행률 표시
            if idx % 300 == 0:
                elapsed = time.time() - start_time
                progress = idx / total_rows * 100
                eta = elapsed * (total_rows - idx) / idx if idx > 0 else 0

                print(f"📈 진행률: {idx:,}/{total_rows:,} ({progress:.1f}%) | "
                      f"경과: {elapsed/60:.1f}분 | 예상 잔여: {eta/60:.1f}분")

            # 각 텍스트 컬럼에서 추출
            for col in text_columns:
                text = str(row.get(col, ''))

                if text and text != 'nan' and len(text.strip()) > 5:
                    try:
                        entities = self.extract_entities_from_text(text)

                        # 가중치 적용
                        weight = 5 if col == 'title' else 1  # 제목 가중치 증가

                        for entity in entities:
                            term = entity['text']
                            label = entity['label']
                            confidence = entity['confidence']

                            # 고품질 용어만 저장
                            if term and len(term) >= 3:
                                all_entities[term] += weight

                                entity_details.append({
                                    'term': term,
                                    'label': label,
                                    'confidence': confidence,
                                    'source': col,
                                    'doc_index': idx,
                                    'weight': weight
                                })

                    except Exception as e:
                        continue

        total_time = time.time() - start_time
        print(f"\n✅ 추출 완료! 총 소요시간: {total_time/60:.1f}분")
        print(f"📊 추출된 고유 의료용어: {len(all_entities):,}개")

        return dict(all_entities), entity_details

print("✅ 개선된 추출기 준비 완료")

print("\n" + "="*50)

# ==========================================
# 3단계: 모델 로드 및 추출 실행
# ==========================================

print("🚀 === 고품질 의료용어 추출 실행 ===")

# 이전에 로드된 모델 활용 (전역변수에서)
if 'model' in globals() and 'tokenizer' in globals() and 'id2label' in globals():
    print("✅ 기존 로드된 모델 사용")
else:
    print("🔄 모델 재로드 중...")
    # 모델 재로드 코드 (필요시)

# 개선된 추출기 생성
refined_extractor = RefinedMedicalNERExtractor(
    model, tokenizer, id2label,
    confidence_threshold=0.75  # 더 높은 신뢰도 요구
)

# 심평원 데이터 추출 실행
print("🎯 고품질 의료용어 추출 중...")
print("⚙️ 설정: 신뢰도 75% 이상, 불용어 제거, 조사 제거")

refined_terms, refined_details = refined_extractor.extract_from_dataframe(
    df, text_columns=['title', 'content']
)

print(f"\n🎊 개선된 추출 결과:")
print(f"📊 총 고품질 의료용어: {len(refined_terms):,}개")
print(f"📝 총 추출 인스턴스: {len(refined_details):,}개")

# ==========================================
# 4단계: 결과 분석 및 출력
# ==========================================

print("\n📈 === 고품질 결과 분석 ===")

# 빈도별 정렬
sorted_refined_terms = sorted(refined_terms.items(), key=lambda x: x[1], reverse=True)

# 상위 100개 출력
print(f"\n🔝 상위 100개 고품질 의료용어:")
for rank, (term, freq) in enumerate(sorted_refined_terms[:100], 1):
    print(f"{rank:2d}. {term:25s} ({freq:3d}회)")

# 라벨별 분포
label_counts = Counter([detail['label'] for detail in refined_details])
print(f"\n🏷️ 라벨별 분포:")
for label, count in label_counts.most_common():
    percentage = count / len(refined_details) * 100
    print(f"  {label:15s}: {count:,}개 ({percentage:.1f}%)")

# 결과 저장
print(f"\n💾 고품질 결과 저장 중...")

refined_csv = "refined_medical_terms.csv"
with open(refined_csv, 'w', encoding='utf-8') as f:
    f.write("rank,term,frequency,avg_confidence,main_label\n")

    for rank, (term, freq) in enumerate(sorted_refined_terms, 1):
        term_details = [d for d in refined_details if d['term'] == term]

        if term_details:
            avg_conf = sum(d['confidence'] for d in term_details) / len(term_details)
            main_label = Counter(d['label'] for d in term_details).most_common(1)[0][0]
        else:
            avg_conf = 0.0
            main_label = "Unknown"

        f.write(f"{rank},{term},{freq},{avg_conf:.3f},{main_label}\n")

print(f"✅ 고품질 결과 저장: {refined_csv}")

print("\n🎉 === 고품질 의료용어 추출 완료! ===")
print(f"✅ 불용어 제거, 조사 제거, 고신뢰도 필터링 적용")
print(f"📊 최종 고품질 의료용어: {len(refined_terms):,}개")
print(f"📁 결과 파일: {refined_csv}")

🧹 === 고품질 의료용어 추출 시작 ===
📋 === 의료용어 필터링 규칙 정의 ===
✅ 필터링 규칙 정의 완료
📝 불용어 수: 105개

🤖 === 개선된 NER 추출기 ===
✅ 개선된 추출기 준비 완료

🚀 === 고품질 의료용어 추출 실행 ===
✅ 기존 로드된 모델 사용
🎯 고품질 의료용어 추출 중...
⚙️ 설정: 신뢰도 75% 이상, 불용어 제거, 조사 제거
📊 총 8,413개 문서 처리 시작...
📈 진행률: 0/8,413 (0.0%) | 경과: 0.0분 | 예상 잔여: 0.0분
📈 진행률: 300/8,413 (3.6%) | 경과: 0.1분 | 예상 잔여: 3.6분
📈 진행률: 600/8,413 (7.1%) | 경과: 0.3분 | 예상 잔여: 3.4분
📈 진행률: 900/8,413 (10.7%) | 경과: 0.4분 | 예상 잔여: 3.3분
📈 진행률: 1,200/8,413 (14.3%) | 경과: 0.5분 | 예상 잔여: 3.2분
📈 진행률: 1,500/8,413 (17.8%) | 경과: 0.7분 | 예상 잔여: 3.0분
📈 진행률: 1,800/8,413 (21.4%) | 경과: 0.8분 | 예상 잔여: 2.9분
📈 진행률: 2,100/8,413 (25.0%) | 경과: 0.9분 | 예상 잔여: 2.8분
📈 진행률: 2,400/8,413 (28.5%) | 경과: 1.0분 | 예상 잔여: 2.6분
📈 진행률: 2,700/8,413 (32.1%) | 경과: 1.2분 | 예상 잔여: 2.5분
📈 진행률: 3,000/8,413 (35.7%) | 경과: 1.3분 | 예상 잔여: 2.3분
📈 진행률: 3,300/8,413 (39.2%) | 경과: 1.4분 | 예상 잔여: 2.2분
📈 진행률: 3,600/8,413 (42.8%) | 경과: 1.5분 | 예상 잔여: 2.1분
📈 진행률: 3,900/8,413 (46.4%) | 경과: 1.7분 | 예상 잔여: 1.9분
📈 진행률: 4,200/8,413 (49.9%) | 경과: 1.8분 | 예상 잔여: 1.8분

In [ ]:
# 🔧 고품질 의료용어 추출 (불용어 제거 + 후처리)

print("🧹 === 고품질 의료용어 추출 시작 ===")

import pandas as pd
import torch
import json
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForTokenClassification
import time

# ==========================================
# 1단계: 의료 불용어 및 필터링 규칙 정의
# ==========================================

print("📋 === 의료용어 필터링 규칙 정의 ===")

# 의료 불용어 (너무 일반적이거나 의미없는 용어들)
MEDICAL_STOPWORDS = {
    # 너무 일반적인 의료 용어
    '수술', '수술을', '수술의', '수술에서', '수술시', '수술후',
    '치료', '치료를', '치료의', '치료에', '치료법', '치료제',
    '검사', '검사를', '검사의', '검사에서', '검사시', '검사후',
    '진료', '진료를', '진료의', '진료에서', '진료시', '진료후',
    '처치', '처치를', '처치의', '처치에서', '처치시', '처치후',
    '시술', '시술을', '시술의', '시술에서', '시술시', '시술후',

    # 제형 관련 일반 용어
    '주사제', '경구제', '정제', '캡슐', '시럽', '연고', '겔',
    '외용제', '내복약', '주사약', '정맥주사', '근육주사',

    # 너무 일반적인 해부학 용어
    '피부', '피부의', '뼈', '뼈의', '근육', '근육의',
    '혈관', '혈관의', '신경', '신경의', '조직', '조직의',

    # 행정/절차 용어
    '요양급', '선별급여', '급여', '급여의', '비급여',
    '외래', '외래의', '입원', '입원의', '퇴원', '퇴원의',
    '간호', '간호의', '간병', '간병의',

    # 상태 표현
    '상세불명의', '기타', '기타의', '미상', '미상의',
    '증상', '증상의', '소견', '소견의', '상태', '상태의',

    # 일반 단어
    '환자', '환자의', '질환', '질환의', '질병', '질병의',
    '통증', '통증의', '출혈', '출혈의', '외상', '외상의',
    '종양', '종양의', '암', '암의', '의한', '등의', '관련',
}

# 한국어 조사 패턴
KOREAN_PARTICLES = r'[을를이가에서로으로와과의은는]$|[이가]$|에서$|으로$|와의$|치아에$|유방의$'

# 보존할 고품질 의료용어 패턴
HIGH_QUALITY_PATTERNS = {
    # 구체적인 질병명
    'specific_diseases': [
        r'.*경색.*', r'.*증후군.*', r'.*염.*', r'.*종.*', r'.*암.*',
        r'.*당뇨.*', r'.*고혈압.*', r'.*뇌졸중.*', r'.*폐렴.*',
        r'.*간염.*', r'.*신부전.*', r'.*부정맥.*'
    ],

    # 구체적인 시술/수술명
    'specific_procedures': [
        r'.*이식.*', r'.*치환술.*', r'.*고정술.*', r'.*삽입술.*',
        r'.*절제술.*', r'.*성형술.*', r'.*복원술.*', r'.*재건술.*',
        r'.*투석.*', r'.*요법.*', r'.*면역.*검사.*'
    ],

    # 의료기기/장비
    'medical_devices': [
        r'MRI', r'CT', r'VAD', r'ICD', r'.*내시경.*',
        r'.*카테터.*', r'.*스텐트.*', r'.*임플란트.*'
    ],

    # 구체적 해부학 부위
    'specific_anatomy': [
        r'.*추.*', r'.*관절.*', r'.*판.*장애.*',
        r'구순구개열.*', r'.*심방.*', r'.*심실.*'
    ]
}

def is_high_quality_medical_term(term):
    """고품질 의료용어인지 판단"""

    # 너무 짧거나 긴 용어 제외
    if len(term) < 3 or len(term) > 30:
        return False

    # 불용어 체크
    if term in MEDICAL_STOPWORDS:
        return False

    # 조사 붙은 용어 제외
    if re.search(KOREAN_PARTICLES, term):
        return False

    # 숫자만 있는 용어 제외
    if re.match(r'^[\d\s\-\.]+$', term):
        return False

    # 영문자만 있고 너무 짧은 용어 제외
    if re.match(r'^[A-Za-z\s]{1,3}$', term):
        return False

    # 특수문자가 너무 많은 용어 제외
    if len(re.findall(r'[^\w가-힣\s]', term)) > len(term) * 0.3:
        return False

    # 고품질 패턴 중 하나라도 매치되면 보존
    for category, patterns in HIGH_QUALITY_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, term, re.IGNORECASE):
                return True

    # 추가 의료용어 휴리스틱
    medical_suffixes = ['증', '병', '염', '종', '암', '술', '법', '요법', '검사', '촬영']
    if any(term.endswith(suffix) for suffix in medical_suffixes):
        return True

    # 의료 접두사
    medical_prefixes = ['급성', '만성', '외상성', '감염성', '자가면역']
    if any(term.startswith(prefix) for prefix in medical_prefixes):
        return True

    return False

print("✅ 필터링 규칙 정의 완료")
print(f"📝 불용어 수: {len(MEDICAL_STOPWORDS)}개")

print("\n" + "="*50)

# ==========================================
# 2단계: 개선된 NER 추출기
# ==========================================

print("🤖 === 개선된 NER 추출기 ===")

class RefinedMedicalNERExtractor:
    def __init__(self, model, tokenizer, id2label, confidence_threshold=0.75):
        self.model = model
        self.tokenizer = tokenizer
        self.id2label = id2label
        self.confidence_threshold = confidence_threshold
        self.model.eval()

    def clean_extracted_term(self, term):
        """추출된 용어 정리"""

        # 앞뒤 공백 제거
        term = term.strip()

        # 특수문자 정리
        term = re.sub(r'^[^\w가-힣]+|[^\w가-힣]+$', '', term)

        # 연속된 공백 정리
        term = re.sub(r'\s+', ' ', term)

        # 조사 제거
        term = re.sub(KOREAN_PARTICLES, '', term)

        return term.strip()

    def extract_entities_from_text(self, text, max_length=256):
        """텍스트에서 고품질 의료 개체명 추출"""

        if not text or len(text.strip()) < 3:
            return []

        # 텍스트 전처리
        text = re.sub(r'\s+', ' ', text.strip())
        words = text.split()

        # 길이 제한
        if len(words) > max_length // 3:
            words = words[:max_length // 3]

        try:
            # 토크나이징
            inputs = self.tokenizer(
                words,
                is_split_into_words=True,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=max_length
            )

            # GPU로 이동
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            # 예측
            with torch.no_grad():
                outputs = self.model(**inputs)
                probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predictions = probabilities.argmax(dim=-1)

            # 결과 추출 (개선된 로직)
            entities = []
            current_entity = ""
            current_label = None
            current_confidences = []

            tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            predictions_cpu = predictions[0].cpu().numpy()
            probabilities_cpu = probabilities[0].cpu().numpy()

            for i, (token, pred_id) in enumerate(zip(tokens, predictions_cpu)):

                if token in ['[CLS]', '[SEP]', '[PAD]']:
                    continue

                confidence = probabilities_cpu[i][pred_id]
                label = self.id2label[pred_id]

                # 높은 신뢰도만 처리
                if confidence < self.confidence_threshold:
                    # 현재 개체 저장
                    if current_entity and current_label != 'O':
                        avg_conf = sum(current_confidences) / len(current_confidences)
                        cleaned_term = self.clean_extracted_term(current_entity)

                        if cleaned_term and is_high_quality_medical_term(cleaned_term):
                            entities.append({
                                'text': cleaned_term,
                                'label': current_label,
                                'confidence': avg_conf
                            })

                    current_entity = ""
                    current_label = None
                    current_confidences = []
                    continue

                # 서브워드 처리
                if token.startswith('##'):
                    if current_entity:
                        current_entity += token[2:]
                        current_confidences.append(confidence)
                    continue

                # B- 태그 (새로운 개체)
                if label.endswith('-B'):
                    # 이전 개체 저장
                    if current_entity and current_label != 'O':
                        avg_conf = sum(current_confidences) / len(current_confidences)
                        cleaned_term = self.clean_extracted_term(current_entity)

                        if cleaned_term and is_high_quality_medical_term(cleaned_term):
                            entities.append({
                                'text': cleaned_term,
                                'label': current_label,
                                'confidence': avg_conf
                            })

                    # 새로운 개체 시작
                    current_entity = token
                    current_label = label[:-2]
                    current_confidences = [confidence]

                # I- 태그 (개체 연속)
                elif label.endswith('-I') and current_label == label[:-2]:
                    current_entity += " " + token
                    current_confidences.append(confidence)

                # O 태그 또는 다른 개체
                else:
                    # 이전 개체 저장
                    if current_entity and current_label != 'O':
                        avg_conf = sum(current_confidences) / len(current_confidences)
                        cleaned_term = self.clean_extracted_term(current_entity)

                        if cleaned_term and is_high_quality_medical_term(cleaned_term):
                            entities.append({
                                'text': cleaned_term,
                                'label': current_label,
                                'confidence': avg_conf
                            })

                    current_entity = ""
                    current_label = None
                    current_confidences = []

            # 마지막 개체 저장
            if current_entity and current_label != 'O':
                avg_conf = sum(current_confidences) / len(current_confidences)
                cleaned_term = self.clean_extracted_term(current_entity)

                if cleaned_term and is_high_quality_medical_term(cleaned_term):
                    entities.append({
                        'text': cleaned_term,
                        'label': current_label,
                        'confidence': avg_conf
                    })

            return entities

        except Exception as e:
            print(f"❌ 텍스트 처리 오류: {e}")
            return []

    def extract_from_dataframe(self, df, text_columns=['title', 'content']):
        """데이터프레임에서 고품질 의료용어 대량 추출"""

        all_entities = Counter()
        entity_details = []

        total_rows = len(df)
        print(f"📊 총 {total_rows:,}개 문서 처리 시작...")

        start_time = time.time()

        for idx, row in df.iterrows():

            # 진행률 표시
            if idx % 300 == 0:
                elapsed = time.time() - start_time
                progress = idx / total_rows * 100
                eta = elapsed * (total_rows - idx) / idx if idx > 0 else 0

                print(f"📈 진행률: {idx:,}/{total_rows:,} ({progress:.1f}%) | "
                      f"경과: {elapsed/60:.1f}분 | 예상 잔여: {eta/60:.1f}분")

            # 각 텍스트 컬럼에서 추출
            for col in text_columns:
                text = str(row.get(col, ''))

                if text and text != 'nan' and len(text.strip()) > 5:
                    try:
                        entities = self.extract_entities_from_text(text)

                        # 가중치 적용
                        weight = 5 if col == 'title' else 1  # 제목 가중치 증가

                        for entity in entities:
                            term = entity['text']
                            label = entity['label']
                            confidence = entity['confidence']

                            # 고품질 용어만 저장
                            if term and len(term) >= 3:
                                all_entities[term] += weight

                                entity_details.append({
                                    'term': term,
                                    'label': label,
                                    'confidence': confidence,
                                    'source': col,
                                    'doc_index': idx,
                                    'weight': weight
                                })

                    except Exception as e:
                        continue

        total_time = time.time() - start_time
        print(f"\n✅ 추출 완료! 총 소요시간: {total_time/60:.1f}분")
        print(f"📊 추출된 고유 의료용어: {len(all_entities):,}개")

        return dict(all_entities), entity_details

print("✅ 개선된 추출기 준비 완료")

print("\n" + "="*50)

# ==========================================
# 3단계: 모델 로드 및 추출 실행
# ==========================================

print("🚀 === 고품질 의료용어 추출 실행 ===")

# 이전에 로드된 모델 활용 (전역변수에서)
if 'model' in globals() and 'tokenizer' in globals() and 'id2label' in globals():
    print("✅ 기존 로드된 모델 사용")
else:
    print("🔄 모델 재로드 중...")
    # 모델 재로드 코드 (필요시)

# 개선된 추출기 생성
refined_extractor = RefinedMedicalNERExtractor(
    model, tokenizer, id2label,
    confidence_threshold=0.75  # 더 높은 신뢰도 요구
)

# 심평원 데이터 추출 실행
print("🎯 고품질 의료용어 추출 중...")
print("⚙️ 설정: 신뢰도 75% 이상, 불용어 제거, 조사 제거")

refined_terms, refined_details = refined_extractor.extract_from_dataframe(
    df, text_columns=['title', 'content']
)

print(f"\n🎊 개선된 추출 결과:")
print(f"📊 총 고품질 의료용어: {len(refined_terms):,}개")
print(f"📝 총 추출 인스턴스: {len(refined_details):,}개")

# ==========================================
# 4단계: 결과 분석 및 출력
# ==========================================

print("\n📈 === 고품질 결과 분석 ===")

# 빈도별 정렬
sorted_refined_terms = sorted(refined_terms.items(), key=lambda x: x[1], reverse=True)

# 상위 1000개 출력
print(f"\n🔝 상위 1000개 고품질 의료용어:")
for rank, (term, freq) in enumerate(sorted_refined_terms[:1000], 1):
    print(f"{rank:3d}. {term:30s} ({freq:3d}회)")

# 라벨별 분포
label_counts = Counter([detail['label'] for detail in refined_details])
print(f"\n🏷️ 라벨별 분포:")
for label, count in label_counts.most_common():
    percentage = count / len(refined_details) * 100
    print(f"  {label:15s}: {count:,}개 ({percentage:.1f}%)")

# 결과 저장
print(f"\n💾 고품질 결과 저장 중...")

refined_csv = "refined_medical_terms.csv"
with open(refined_csv, 'w', encoding='utf-8') as f:
    f.write("rank,term,frequency,avg_confidence,main_label\n")

    for rank, (term, freq) in enumerate(sorted_refined_terms, 1):
        term_details = [d for d in refined_details if d['term'] == term]

        if term_details:
            avg_conf = sum(d['confidence'] for d in term_details) / len(term_details)
            main_label = Counter(d['label'] for d in term_details).most_common(1)[0][0]
        else:
            avg_conf = 0.0
            main_label = "Unknown"

        f.write(f"{rank},{term},{freq},{avg_conf:.3f},{main_label}\n")

print(f"✅ 고품질 결과 저장: {refined_csv}")

print("\n🎉 === 고품질 의료용어 추출 완료! ===")
print(f"✅ 불용어 제거, 조사 제거, 고신뢰도 필터링 적용")
print(f"📊 최종 고품질 의료용어: {len(refined_terms):,}개")
print(f"📁 결과 파일: {refined_csv}")

🧹 === 고품질 의료용어 추출 시작 ===
📋 === 의료용어 필터링 규칙 정의 ===
✅ 필터링 규칙 정의 완료
📝 불용어 수: 105개

🤖 === 개선된 NER 추출기 ===
✅ 개선된 추출기 준비 완료

🚀 === 고품질 의료용어 추출 실행 ===
✅ 기존 로드된 모델 사용
🎯 고품질 의료용어 추출 중...
⚙️ 설정: 신뢰도 75% 이상, 불용어 제거, 조사 제거
📊 총 8,413개 문서 처리 시작...
📈 진행률: 0/8,413 (0.0%) | 경과: 0.0분 | 예상 잔여: 0.0분
📈 진행률: 300/8,413 (3.6%) | 경과: 0.1분 | 예상 잔여: 3.6분
📈 진행률: 600/8,413 (7.1%) | 경과: 0.3분 | 예상 잔여: 3.4분
📈 진행률: 900/8,413 (10.7%) | 경과: 0.4분 | 예상 잔여: 3.3분
📈 진행률: 1,200/8,413 (14.3%) | 경과: 0.5분 | 예상 잔여: 3.2분
📈 진행률: 1,500/8,413 (17.8%) | 경과: 0.7분 | 예상 잔여: 3.0분
📈 진행률: 1,800/8,413 (21.4%) | 경과: 0.8분 | 예상 잔여: 2.9분
📈 진행률: 2,100/8,413 (25.0%) | 경과: 0.9분 | 예상 잔여: 2.7분
📈 진행률: 2,400/8,413 (28.5%) | 경과: 1.0분 | 예상 잔여: 2.6분
📈 진행률: 2,700/8,413 (32.1%) | 경과: 1.2분 | 예상 잔여: 2.5분
📈 진행률: 3,000/8,413 (35.7%) | 경과: 1.3분 | 예상 잔여: 2.3분
📈 진행률: 3,300/8,413 (39.2%) | 경과: 1.4분 | 예상 잔여: 2.2분
📈 진행률: 3,600/8,413 (42.8%) | 경과: 1.5분 | 예상 잔여: 2.1분
📈 진행률: 3,900/8,413 (46.4%) | 경과: 1.7분 | 예상 잔여: 1.9분
📈 진행률: 4,200/8,413 (49.9%) | 경과: 1.8분 | 예상 잔여: 1.8분